# 1. Setup & Bus Data Load

Import required modules and packages.

In [ ]:
# import os so that environment variables can be accessed (for database password, etc.)
import os

# import math for mathematical functions
import math

# import pandas and numpy for data analysis
import pandas as pd
import numpy as np

# import transform_data function for transforming data into segment format
from transform_data import transform_data

# import convert_timestamp for various timestamp conversion functions
import convert_timestamp

# import matplotlib & seaborn for plotting and visualisaion
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# import from sklearn for machine learning
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn import metrics

# import pickle so that models can be saved to file
import pickle

Set the max number of columns & rows to display.

In [ ]:
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 500)

Load the bus data from file.

In [ ]:
df = pd.read_csv('/data_analytics/data/all_routes_jul_aug.csv', sep=";", na_values=['\\N'])

Perform a check to see how many rows and columns are in the file.

In [ ]:
rows = df.shape[0]
cols = df.shape[1]
print()
print("Before any data cleaning, the CSV file contains", rows, "rows and", cols, "columns.")
print()

Because the data is so large, I will initially just perform data analysis for August data:

In [ ]:
df = df.loc[(df['dayofservice'] >= '2018-08-01 00:00:00') & (df['dayofservice'] <= '2018-08-31 00:00:00')]
# reset the index
df = df.reset_index(drop=True)

# 2. Initial Checks on the Bus Data

## 2.1 Check for Duplicate Rows & Columns

In [ ]:
print()
print('Duplicate rows:', df.duplicated()[df.duplicated() == True].shape[0])
print('Duplicate columns:',df.columns.size - df.columns.unique().size)

There are no duplicate rows or columns in the bus data.

## 2.2 Check for Null Features

In [ ]:
df.describe().T

There are no null features in the bus data.

## 2.3 Assign Features as Continuous or Categorical

Assign categorical and continous features, and update the type of all categorical features to 'category'.

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['dayofservice', 'tripid', 'lineid', 'routeid', 'direction', 'progrnumber', 'stoppointid']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

## 2.4 Check for Constant Categorical Features

In [ ]:
# Print details for the categorical columns
df[categorical_columns].describe().T

There are no constant features to be droppped.

## 2.5 Check for Constant Continuous Features

In [ ]:
# Print details for the continuous columns
df[continuous_columns].describe().T

There are no constant features to be droppped.

# 3. Initial Checks for Missing Data

## 3.1 Categorical Features

Print details for the categorical columns:

In [ ]:
df[categorical_columns].describe().T

There is a full count for all categorical features.

## 3.2 Continuous Features

Print details for the continuous columns:

In [ ]:
df[continuous_columns].describe().T

There are some rows missing data for **actualtime_arr_trip** and **actualtime_dep_trip**. This will be reviewed if these features are used in the future, currently they are not carried across when data is transformed.

# 4. Transform the Bus Data

Bus data must be transformed so that each row of data holds information on one journey segment. This is done by calling the *transform_data* function:

In [ ]:
df = transform_data(df)

In [ ]:
df.head(5)

## 4.1 Check for Missing Data

First re-assign the transformed data as continuous or categorical:

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['dayofservice', 'tripid', 'lineid', 'routeid', 'direction',  \
                         'progrnumber_first', 'stoppointid_first', \
                          'progrnumber_next', 'stoppointid_next']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

Then check for missing data:

In [ ]:
# Print details for the categorical columns
df[categorical_columns].describe().T

In [ ]:
# Print details for the continuous columns
df[continuous_columns].describe().T

There are some rows with missing data, because the amount of missing rows is quite low, and because imputation would be difficult, these rows will be dropped.

## 5.2 Drop Rows with Missing Data

Drop rows where *stoppointid_first* or *stoppointid_next* is null:

In [ ]:
df = df[pd.notnull(df['stoppointid_first'])]

In [ ]:
df = df[pd.notnull(df['stoppointid_next'])]

# 6. Import Weather Data

Weather data is loaded from the database:

In [ ]:
df_weather = pd.read_csv('/data_analytics/data/weather.csv', sep=",", na_values=['\\N'])

In [ ]:
df_weather.head(5)

## 6.1 Check for Duplicate Rows & Columns

In [ ]:
print()
print('Duplicate rows:', df_weather.duplicated()[df_weather.duplicated() == True].shape[0])
print('Duplicate columns:',df_weather.columns.size - df_weather.columns.unique().size)

There are no duplicate rows or columns so nothing needs to be dropped here.

## 6.2 Assign Features as Continuous or Categorical

First check the data types of all rows after the file import:

In [ ]:
df_weather.dtypes

Assign categorical and continuous features:

In [ ]:
# Select columns containing categorical data
categorical_columns = df_weather[['record_date', 'irain', 'itemp', 'iwb']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df_weather[column] = df_weather[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df_weather.select_dtypes(['float64', 'int64']).columns

## 6.3 Check for Missing Data, Constant Features, etc.

In [ ]:
# Print details for the categorical columns
df_weather[categorical_columns].describe().T

**itemp** and **iwb** are constant columns so can be dropped.

In [ ]:
# Print details for the categorical columns
df_weather[continuous_columns].describe().T

Investigate rows with missing data for rain:

In [ ]:
# select all rows where irain is not 0
df_weather.loc[df_weather['irain'] != 0]

There are only two rows where irain is not zero, these rows correspond to missing values for rain.

In [ ]:
# select other rows around the missing values
df_weather[6220:6240]

Given that there is no rain for the rest of the day, and given the high (for Ireland) temperature on the day, I think it's safe to replace the missing rain values with 0.

I will then drop the feature **irain** as it provides no useful information.

## 6.4 Replace Missing Weather Data

In [ ]:
# replace rain with 0 where irain is not 0
df_weather['rain'].loc[df_weather['irain'] != 0] = 0

In [ ]:
# check that values are updated
df_weather.loc[df_weather['irain'] != 0]

## 6.5 Drop Constant Weather Features

In [ ]:
df_weather = df_weather.drop(columns=['irain', 'itemp', 'iwb'])

# 7. Combine Bus and Weather Data

## 7.1 Split Timestamp for Weather Data

To merge the data, timestamps must be split into month, day and hour.

New features are added:

In [ ]:
df_weather['month'] = df_weather['record_date'].map(lambda x: convert_timestamp.timestamp_to_month_weather(x))

In [ ]:
df_weather['day'] = df_weather['record_date'].map(lambda x: convert_timestamp.timestamp_to_day_weather(x))

In [ ]:
df_weather['hour'] = df_weather['record_date'].map(lambda x: convert_timestamp.timestamp_to_hour_weather(x))

New features are updated to be categorical:

In [ ]:
# Select columns containing categorical data
categorical_columns = df_weather[['record_date', 'month', 'day', 'hour']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df_weather[column] = df_weather[column].astype('category')

In [ ]:
df_weather.head(5)

## 7.2 Split Timestamp for Bus Data

New features are added:

In [ ]:
df['month'] = df.apply (lambda row: convert_timestamp.timestamp_to_month_bus(row['dayofservice'], \
                                                                                   row['actualtime_arr_stop_first']), axis=1)

In [ ]:
df['day'] = df.apply (lambda row: convert_timestamp.timestamp_to_day_bus(row['dayofservice'], \
                                                                               row['actualtime_arr_stop_first']), axis=1)

In [ ]:
df['hour'] = df['actualtime_arr_stop_first'].map(lambda x: convert_timestamp.timestamp_to_hour_bus(x))

## 7.3 Merge the Dataframes

In [ ]:
df = pd.merge(df, df_weather,  how='left', left_on=['month','day', 'hour'],\
                     right_on = ['month','day', 'hour'])

Check that there are no rows missing weather data:

In [ ]:
df[df.rain.isnull()]

Update the data types for the new dataframe:

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['dayofservice', 'tripid', 'lineid', 'routeid', 'direction',  \
                         'progrnumber_first', 'stoppointid_first', 'progrnumber_next', 'stoppointid_next',\
                          'month', 'day', 'hour', 'record_date']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

In [ ]:
df.head(5)

# 8. Create New Features

## 8.1 Day of Week Feature

The new feature is added:

In [ ]:
df['day_of_week'] = df['dayofservice'].map(lambda x: convert_timestamp.timestamp_to_day_of_week(x))

## 8.2 Weekend/Weekday Feature

The new feature is added:

In [ ]:
df['weekday'] = df['dayofservice'].map(lambda x: convert_timestamp.timestamp_to_weekday_weekend(x))

## 8.3 Bank Holiday Feature

Make list of bank holidays for 2018 (based on https://www.officeholidays.com/countries/ireland/2018):

In [ ]:
holidays = ['2018-01-01 00:00:00', '2018-03-19 00:00:00', '2018-04-02 00:00:00', '2018-05-07 00:00:00', '2018-06-04 00:00:00',\
           '2018-08-06 00:00:00', '2018-10-29 00:00:00', '2018-12-25 00:00:00', '2018-12-26 00:00:00']

The new feature is added:

In [ ]:
df['bank_holiday'] = df['dayofservice'].map(lambda x: convert_timestamp.timestamp_to_bank_holiday(x, holidays))

## 8.4 Time Difference Feature

The new feature is added:

In [ ]:
df['time_diff'] = df['actualtime_arr_stop_next'] - df['actualtime_arr_stop_first']

## 8.5 Update Data Types for the Features

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['dayofservice', 'tripid', 'lineid', 'routeid', 'direction',  \
                         'progrnumber_first', 'stoppointid_first', 'progrnumber_next', 'stoppointid_next',\
                          'month', 'day', 'hour', 'record_date', 'day_of_week', 'weekday', 'bank_holiday']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

In [ ]:
df.dtypes

# 9. Data Integrity Checks

## 9.1 Time Difference Checks

Check for rows where the time difference is less than or equal to zero:

In [ ]:
df.loc[df['time_diff'] <= 0]

There are 959 rows that fail this test. These will be dropped:

In [ ]:
df = df.loc[df['time_diff'] > 0]

## 9.2 Segment Checks

We need to check if we are missing data for any segments.

In [ ]:
# make a list of all segments
segments = [(226,228),(228,229),(229,227),(227,230),(230,231),(231,1641),(1641,1642),(1642,213),(213,214),(214,4432),(4432,119),(119,44),(44,7603),(7603,45),(45,46),(46,47),(47,48),(48,49),(49,51),(51,52),(52,265),(265,271),(271,340),(340,350),(350,351),(351,352),(352,353),(353,354),(354,355),(355,356),(356,357),(357,390),(390,372),(372,373),(373,374),(374,375),(375,2804),(2804,376),(376,377),(377,378),(378,380),(381,382),(382,4451),(4451,383),(383,384),(384,385),(385,387),(387,388),(388,389),(389,393),(393,371),(371,391),(391,392),(392,395),(395,396),(396,397),(397,398),(398,399),(399,400),(400,319),(319,278),(278,10),(10,12),(12,14),(14,15),(15,17),(17,18),(18,19),(19,21),(21,7602),(7602,85),(85,203),(203,204),(204,205),(205,1620),(1620,220),(220,221),(221,222),(222,223),(223,224),(224,225),(225,226),(278,7615),(6122,35),(35,36),(36,37),(37,38),(38,39),(39,40),(40,41),(41,42),(42,43),(43,44),(52,261),(261,271),(271,404),(404,747),(747,842),(842,845),(845,846),(846,847),(847,848),(848,849),(849,850),(850,851),(851,852),(852,853),(853,854),(854,855),(855,856),(856,857),(857,858),(858,859),(859,860),(860,861),(861,862),(862,3014),(3014,3015),(3015,3016),(3016,3017),(3017,3018),(3018,3019),(3019,3020),(3020,3021),(3021,3022),(3022,441),(441,442),(442,443),(443,444),(444,445),(445,4779),(4779,7057),(7057,4847),(4847,4848),(449,450),(450,3181),(3181,451),(451,447),(447,5045),(5045,4460),(4460,453),(453,454),(454,4475),(4475,3023),(3023,3024),(3024,3025),(3025,3026),(3026,3027),(3027,3028),(3028,3029),(3029,3030),(3030,3031),(3031,875),(875,876),(876,878),(878,879),(879,880),(880,881),(881,882),(882,883),(883,884),(884,885),(885,886),(886,887),(887,903),(903,904),(904,906),(906,907),(907,908),(908,909),(909,786),(786,793),(793,320),(320,278),(7602,22),(22,23),(23,24),(24,25),(25,27),(27,28),(28,29),(29,4680),(4680,31),(31,32),(32,33),(33,7270),(7270,6122),(324,6235),(6235,327),(327,6016),(6016,6017),(6017,7192),(7192,4683),(4683,107),(107,6125),(6125,7092),(7092,110),(110,112),(112,113),(113,114),(114,115),(115,37),(39,116),(116,117),(117,118),(118,119),(261,270),(270,335),(335,4521),(4521,1935),(1935,1937),(1937,1938),(1938,1939),(1939,1940),(1940,1941),(1941,1942),(1942,1943),(1943,1944),(1944,1945),(1945,1946),(1946,1947),(1947,1948),(1948,1949),(1949,1950),(1950,1951),(1951,1952),(1952,1953),(1953,1954),(1954,1955),(1955,1956),(1956,1957),(1957,1958),(1958,1959),(1959,1960),(1960,4379),(4379,1963),(1963,4549),(4549,1964),(1964,4674),(4674,4663),(4663,4664),(4664,2160),(2160,2161),(2161,4665),(4665,4666),(4666,4667),(4667,4668),(4668,4677),(4677,2136),(2136,2137),(2137,7201),(7201,2138),(2138,2139),(2139,2140),(2140,2141),(2141,2142),(2142,2143),(2143,2145),(2145,4670),(4670,2165),(2165,2166),(2166,4545),(4545,4552),(4552,4554),(4554,7229),(2166,7229),(4554,7230),(7229,4617),(4617,4555),(4555,4576),(4576,3418),(3418,3419),(3419,2374),(2374,2146),(2146,2147),(2147,2148),(2148,2149),(2149,2150),(2150,2151),(2151,2152),(2152,2153),(2153,2154),(2154,6245),(6245,4671),(4671,4672),(4672,3465),(3465,4673),(4673,2172),(2172,2159),(2159,4409),(4409,4550),(4550,4551),(4551,1965),(1965,1966),(1966,1975),(1975,1977),(1977,1978),(1978,1980),(1980,1981),(1981,1982),(1982,4406),(4406,1984),(1984,1985),(1985,1986),(1986,1987),(1987,1988),(1988,1989),(1989,1990),(1990,1992),(1992,1993),(1993,1994),(1994,1995),(1995,1996),(1996,1997),(1997,1998),(1998,2001),(2001,2002),(2002,1359),(1359,319),(319,281),(281,7),(7,11),(11,14),(85,86),(86,87),(87,88),(88,89),(89,27),(4680,90),(90,91),(91,92),(92,93),(93,94),(94,102),(102,6239),(6239,104),(104,6241),(6241,105),(105,6253),(6253,7191),(7191,6238),(6238,6182),(6182,322),(322,323),(323,6234),(6234,332),(7229,3418),(1998,1999),(1999,2001),(21,85),(4555,3418),(7230,4617),(248,249),(249,250),(250,251),(251,252),(252,255),(255,234),(234,235),(235,236),(236,1190),(1190,1191),(1191,1192),(1192,1193),(1193,1194),(1194,1195),(1195,664),(664,665),(665,666),(666,667),(667,668),(668,614),(614,615),(615,616),(616,617),(617,618),(618,619),(619,675),(675,4415),(4415,301),(301,336),(336,7581),(7581,1283),(1283,7579),(7579,1285),(1285,1016),(1016,1017),(1017,1018),(1018,1019),(1019,1020),(1020,1076),(1076,1077),(1077,1078),(1078,1079),(1079,1080),(1080,1081),(1081,2914),(2914,2989),(2989,2990),(2990,1042),(1042,1043),(1043,1044),(1044,1045),(1045,4710),(4710,1046),(1046,1047),(1047,1048),(1048,1049),(1049,4333),(4333,4334),(4334,4712),(4712,2853),(2853,2854),(2854,2855),(2855,2856),(2856,2857),(2857,2858),(2858,2859),(2859,2860),(2860,2861),(2861,2862),(2862,2863),(2863,2864),(2864,4486),(4486,2865),(2865,2866),(2866,6044),(6041,2826),(2826,2842),(2842,2843),(2843,2844),(2844,2845),(2845,2846),(2846,2847),(2847,4397),(4397,2848),(2848,2849),(2849,2850),(2850,2851),(2851,2852),(2852,4707),(4707,4708),(4708,4336),(4336,1051),(1051,1052),(1052,1053),(1053,1054),(1054,1036),(1036,4709),(4709,1037),(1037,1038),(1038,2998),(2998,2999),(2999,3000),(3000,3001),(3001,3002),(3002,1166),(1166,1167),(1167,1168),(1168,1169),(1169,1170),(1170,1069),(1069,1070),(1070,1071),(1071,4528),(4528,1072),(1072,7577),(7577,1353),(1353,1354),(1354,7578),(7578,7582),(7582,297),(297,496),(496,497),(497,515),(515,516),(516,4384),(4384,519),(519,521),(521,522),(522,523),(523,669),(669,670),(670,671),(671,672),(672,4382),(4382,1185),(1185,1186),(1186,1187),(1187,1188),(1188,216),(216,217),(217,242),(242,243),(243,244),(244,245),(245,246),(246,248),(243,254),(254,244),(6318,6319),(6319,7246),(7246,6320),(6320,4594),(4594,4595),(4595,4596),(4596,4563),(4563,1218),(1218,1270),(1270,1272),(1272,1273),(1273,1274),(1274,1275),(1275,1276),(1276,1277),(1277,1219),(1219,1220),(1220,1221),(1221,664),(301,4495),(4495,7581),(1081,1082),(1082,1083),(1083,1085),(1085,1121),(1121,1122),(1122,1123),(1123,1124),(1124,1125),(1125,1127),(1127,1130),(1130,1140),(1140,1141),(1141,1142),(1142,1143),(1143,1144),(1144,1145),(1145,4869),(4869,4870),(4870,3007),(3007,6283),(6283,6282),(6282,6335),(6335,6326),(6326,4886),(4886,4887),(4887,1150),(1150,1151),(1151,1152),(1152,1153),(1153,1154),(1154,946),(946,1155),(1155,1157),(1157,1158),(1158,1159),(1159,1160),(1160,1161),(1161,1162),(1162,1163),(1163,1164),(1164,1165),(1165,1166),(7582,299),(299,497),(4382,1196),(1196,1197),(1197,1198),(1198,1199),(1199,1200),(1200,4385),(4385,1201),(1201,1202),(1202,1203),(1203,6115),(6115,4593),(4593,7516),(7516,6079),(6079,6315),(6315,6316),(6316,7236),(7236,7245),(7245,6317),(7347,3669),(3669,7349),(7349,1631),(1631,1632),(1632,5053),(5053,1633),(1633,1634),(1634,1635),(1635,1636),(1636,1637),(1637,1638),(1638,1639),(1639,1640),(1640,231),(231,232),(232,233),(233,234),(236,237),(237,213),(52,262),(262,270),(270,336),(336,1279),(1279,1282),(1282,4456),(4456,1284),(1284,1285),(1285,1287),(1287,1288),(1288,1289),(1289,1290),(1290,1291),(1291,1292),(1292,1293),(1293,1294),(1294,1295),(1295,1296),(1296,1297),(1297,1298),(1298,1299),(1299,1300),(1300,1301),(1301,1302),(1302,1303),(1303,1304),(1304,1305),(1305,1320),(1320,1321),(1321,1322),(1322,1323),(1323,1324),(1324,7068),(7068,2966),(2966,2967),(2967,2968),(2968,2969),(2969,2970),(2970,2971),(2971,5051),(5051,5052),(5052,2975),(208,209),(209,210),(210,211),(211,212),(212,213),(5171,2976),(2976,2977),(2977,2978),(2978,2979),(2979,2980),(2980,2981),(2981,2991),(2991,2992),(2992,1325),(1325,1326),(1326,1327),(1327,1328),(1328,1329),(1329,1330),(1330,1331),(1331,1332),(1332,1333),(1333,1334),(1334,7293),(7293,1335),(1335,1336),(1336,1337),(1337,1338),(1338,1339),(1339,1340),(1340,1341),(1341,1342),(1342,1343),(1343,1344),(1344,1345),(1345,1347),(1347,1348),(1348,1349),(1349,1350),(1350,1352),(1352,1353),(1353,1355),(1355,1357),(1357,1359),(1359,320),(278,8),(8,12),(205,215),(215,216),(217,218),(218,219),(219,220),(220,1622),(1622,1623),(1623,1624),(1624,1625),(1625,1626),(1626,1627),(1627,1628),(1628,1629),(1629,1630),(1630,7347),(7391,493),(493,494),(494,495),(495,400),(400,7588),(7588,7392),(7392,312),(312,1444),(1444,1445),(1445,7078),(7078,1449),(1449,1450),(1450,1451),(1451,2191),(2191,2192),(2192,2193),(2193,2194),(2194,2195),(2195,2196),(2196,2197),(2197,2198),(2198,2200),(2200,2201),(2201,7239),(7239,4401),(4401,2213),(2213,2214),(2214,2215),(2215,2216),(2216,4599),(4599,2217),(2217,2218),(2218,2219),(2219,2220),(2220,2221),(2221,2223),(2223,2224),(2224,3886),(3886,3887),(3887,3888),(3888,3890),(3890,3891),(3891,3949),(3949,3950),(3950,2229),(2229,2230),(2230,2231),(2231,2233),(2233,2232),(2232,2234),(2234,2235),(2235,2236),(2236,2238),(2238,5056),(5056,2239),(2239,2241),(2241,2242),(2242,2243),(2243,2244),(2244,2245),(2245,2246),(2246,2247),(2247,2248),(2248,2249),(2249,2250),(2250,2251),(2251,1472),(1472,4408),(4408,1473),(1473,1474),(1474,1476),(1476,7453),(7453,1478),(1478,1479),(1479,315),(315,406),(406,4530),(4530,748),(748,7390),(400,346),(346,317),(317,312),(4401,2202),(2202,2203),(2203,2204),(2204,4888),(4888,4798),(4798,4715),(4795,2686),(2686,7510),(7510,2207),(2207,2208),(2208,2210),(2210,2211),(2211,2241),(406,2810),(2810,7387),(1218,1258),(1258,1259),(1259,1260),(1260,6130),(6130,1262),(1262,4692),(4692,1245),(1245,4693),(4693,1254),(1254,1255),(1255,1256),(1256,4694),(4694,1249),(1249,1247),(1247,1248),(1248,1273),(675,1500),(1500,302),(302,4495),(4495,4522),(4522,1934),(1934,2310),(2310,2311),(2311,2312),(2312,2313),(2313,2314),(2314,2315),(2315,2094),(2094,1406),(1406,1407),(1407,1409),(1409,2095),(2095,2096),(2096,2097),(2097,2099),(2099,2101),(2101,2102),(2102,2103),(2103,2332),(2332,2333),(2333,2334),(2334,4662),(4662,2335),(2335,2336),(2336,2337),(2337,2339),(2339,2340),(2340,2601),(2601,4446),(4446,2603),(2603,2557),(2557,4436),(4436,5008),(5008,4640),(4640,4347),(4347,2349),(2349,2351),(2351,2621),(2621,2622),(2622,2623),(2623,2624),(2624,4441),(4441,2626),(2626,2669),(2669,4641),(4641,4642),(4642,2355),(1260,1262),(400,4522),(2353,4643),(4643,4644),(4644,4645),(4645,4440),(4440,2628),(2628,2629),(2629,6095),(6095,7062),(7062,6075),(6075,2631),(2631,2359),(2359,2360),(2360,4348),(4348,4646),(4646,4647),(4647,2346),(2346,4435),(4435,2632),(2632,2633),(2633,2369),(2369,2370),(2370,2371),(2371,2372),(2372,2373),(2373,2377),(2377,2378),(2378,2183),(2183,2184),(2184,2185),(2185,7043),(7043,1424),(1424,2186),(2186,2187),(2187,2188),(2188,2189),(2189,1436),(1436,3952),(3952,2190),(2190,4434),(4434,2379),(2379,2380),(2380,2382),(2382,5099),(5099,2383),(2383,2002),(2002,1358),(1358,298),(298,496),(1201,4695),(4695,1243),(1243,4696),(4696,4697),(4697,1250),(1250,4698),(4698,4383),(4383,4699),(4699,4388),(4388,4700),(4700,1266),(1266,1267),(1267,1268),(1268,1269),(1269,1265),(1265,6115),(4593,4595),(2377,2431),(2431,2432),(2432,2433),(2433,2327),(2327,2328),(2328,2329),(2329,2330),(2330,2331),(2331,1424),(621,289),(289,496),(496,515),(523,525),(525,526),(526,527),(527,528),(528,529),(529,530),(530,531),(531,709),(709,532),(532,533),(533,534),(534,535),(535,536),(536,537),(537,538),(538,539),(539,540),(540,541),(541,542),(542,543),(543,544),(544,545),(545,546),(546,547),(547,548),(548,549),(549,550),(550,551),(551,552),(552,553),(553,554),(554,555),(555,557),(557,558),(558,559),(559,560),(560,562),(562,563),(563,564),(564,565),(565,566),(566,567),(566,707),(707,708),(708,731),(731,710),(710,711),(711,712),(712,713),(713,714),(714,715),(715,716),(716,717),(717,718),(718,719),(719,720),(720,721),(721,693),(693,585),(567,568),(568,569),(569,570),(570,571),(571,572),(572,573),(573,574),(574,575),(575,576),(576,577),(577,579),(579,580),(580,581),(581,582),(582,583),(583,584),(584,585),(585,586),(586,587),(587,588),(588,589),(589,590),(590,591),(591,592),(592,593),(593,594),(594,595),(595,596),(596,597),(597,598),(598,599),(599,600),(600,601),(601,602),(602,603),(603,604),(604,605),(605,606),(606,607),(607,608),(608,609),(609,610),(610,611),(611,612),(612,613),(613,614),(675,620),(620,7569),(546,910),(910,911),(911,912),(912,913),(913,914),(914,938),(938,940),(940,941),(941,942),(942,943),(943,3599),(3599,3600),(3600,3601),(3601,3602),(3602,3604),(3604,3616),(3616,3618),(3618,3619),(3619,3620),(3620,3622),(3622,3623),(3623,4387),(3624,3587),(3587,3588),(3588,3590),(3590,3589),(3589,3591),(3591,3592),(3592,3593),(3593,3594),(3594,3595),(3595,3596),(3596,3597),(3597,3598),(3598,4465),(4465,944),(944,945),(945,947),(947,948),(948,949),(949,928),(928,929),(929,930),(930,931),(931,587),(292,1171),(1171,1172),(1172,1173),(1173,1174),(1174,1175),(1175,15),(1630,3670),(3670,3671),(3671,3672),(3672,3674),(3674,3675),(3675,3676),(3676,3677),(3677,3678),(3678,3679),(3679,3711),(3711,3712),(3712,3713),(3713,3714),(3714,3715),(3715,5175),(5175,3755),(3755,3756),(3756,3757),(3757,3758),(3758,3759),(3759,3760),(3760,3761),(3761,3762),(3762,3763),(3763,3764),(3764,3765),(3765,3766),(3766,3767),(3767,3768),(3768,3770),(3770,3769),(3769,3771),(3771,3772),(3772,3773),(3773,3774),(3774,7172),(7172,3775),(3775,3777),(3777,3778),(3778,7690),(7690,3781),(3781,3782),(3782,3783),(3783,3784),(3784,3785),(3785,3786),(3786,3787),(3787,3788),(3788,3789),(3789,3790),(3790,3791),(3791,3792),(3792,3793),(3793,3794),(3794,3795),(3795,3797),(3797,3798),(3798,3799),(3799,6076),(6076,6077),(6077,3569),(3569,3571),(3571,3801),(3801,3802),(3802,3805),(3805,7175),(7175,3806),(3806,3807),(3807,3808),(3808,3809),(3809,7174),(6048,5064),(5064,7173),(7173,3811),(3811,3812),(3812,3664),(3664,3813),(3813,3814),(3814,3815),(3815,3816),(3816,3566),(3566,3567),(3567,7292),(7292,3818),(3818,3819),(3819,3820),(3820,3822),(3822,3823),(3823,3824),(3824,3565),(3565,3825),(3825,3826),(3826,3827),(3827,3828),(3828,3829),(3829,3831),(3831,3832),(3832,3833),(3833,3834),(3834,3835),(3835,3836),(3836,3837),(3837,3838),(3838,3840),(3840,3842),(3842,3843),(3843,3844),(3844,3845),(3845,3846),(3846,3848),(3848,3849),(3849,3850),(3850,3851),(3851,3852),(3852,3853),(3853,3854),(3854,3855),(3855,3856),(3856,3857),(3857,3858),(3858,3859),(3859,3860),(3860,3861),(3861,3862),(3862,3863),(3863,3748),(3748,6050),(6050,3749),(3749,3750),(3750,3752),(3752,3751),(3751,3689),(3689,3691),(3691,3692),(3692,3694),(3694,3695),(3695,3696),(3696,3697),(3697,3698),(3698,3699),(3699,3885),(3885,1631),(52,277),(277,292),(7592,3815),(7340,783),(783,784),(784,785),(785,786),(793,7586),(7586,7587),(7587,7588),(7588,329),(329,1443),(1443,1444),(1445,1647),(1647,1648),(1648,1649),(1649,1911),(1911,802),(802,1654),(1654,1655),(1655,1656),(1656,1657),(1657,4458),(4458,1658),(1658,1659),(1659,1660),(1660,1661),(1661,1662),(1662,1664),(1664,1665),(1665,1666),(1666,1667),(1667,1668),(1668,1669),(1669,1670),(1670,1671),(1671,1672),(1672,1673),(1673,1674),(1674,1675),(1675,7380),(7380,1676),(1676,1517),(1517,1678),(1678,1679),(1679,4520),(4520,4825),(4825,7233),(7233,4573),(4573,4893),(4893,4894),(4894,4895),(4895,7217),(7217,7227),(7227,6172),(6172,4493),(4493,4362),(4362,2959),(4747,1882),(1882,7379),(7379,4903),(4903,7219),(7219,7218),(7218,7031),(7031,4897),(4897,4898),(4898,4574),(4574,4575),(4575,1680),(1680,1681),(1681,1682),(1682,1683),(1683,1684),(1684,1685),(1685,1686),(1686,1687),(1687,1688),(1688,1689),(1689,1690),(1690,1691),(1691,1692),(1692,1693),(1693,1694),(1694,1695),(1695,1696),(1696,1697),(1697,1698),(1698,1699),(1699,1700),(1700,1701),(1701,1702),(1702,1703),(1703,1704),(1704,1705),(1705,1706),(1706,1707),(1707,1708),(1708,1709),(1709,1528),(1528,1710),(1710,1711),(1711,1712),(1712,1713),(1713,1714),(1714,1715),(1715,7453),(315,404),(747,843),(843,845),(846,7340),(315,313),(7333,783),(793,274),(274,2),(2,192),(192,795),(795,796),(796,57),(57,58),(58,59),(59,60),(60,1802),(1802,1795),(1795,1806),(1806,1660),(1666,1807),(1807,7167),(7167,1808),(1808,2961),(2961,4901),(4901,4902),(4902,1672),(1675,1849),(1849,1851),(1851,1813),(1813,1814),(1814,1816),(1816,1817),(1817,1818),(1818,1819),(1819,1822),(1822,4994),(4994,4995),(4995,4996),(4996,1826),(1826,1827),(1827,7382),(7382,7381),(7381,4742),(4742,4743),(4743,4744),(4744,4745),(1816,6171),(6171,4974),(4974,4976),(4976,7126),(7126,7124),(7124,7343),(7343,4980),(4980,2289),(2289,1822),(4745,4989),(4989,4990),(4990,4991),(4991,4992),(4992,1719),(1719,1828),(1828,1829),(1829,4997),(4997,4998),(4998,4999),(4999,1833),(1833,1834),(1834,1835),(1835,1836),(1836,1837),(1837,2338),(2338,1840),(1840,1841),(1841,4314),(4314,1687),(1689,4899),(4899,4900),(4900,4746),(4746,1845),(1845,7166),(7166,1847),(1847,1696),(1703,1905),(1905,1804),(1804,1803),(1803,77),(77,78),(78,79),(79,80),(80,82),(82,817),(817,818),(818,819),(819,265),(265,6059),(6059,404),(747,748),(748,749),(749,750),(750,751),(751,752),(752,753),(753,7370),(7370,7339),(1834,1820),(1820,2468),(2468,7319),(7319,7342),(7342,7125),(7125,7344),(7344,7334),(7334,6274),(6274,1838),(1838,2338),(848,2795),(2795,756),(756,757),(757,758),(758,759),(759,760),(760,761),(761,762),(762,763),(763,2007),(2007,2008),(2008,2009),(2009,2010),(2010,435),(435,2011),(2011,2012),(2012,3322),(3322,3563),(3563,444),(4848,4350),(4350,5143),(5143,2837),(2837,2838),(2838,2839),(2839,2840),(2840,2845),(2852,4987),(4987,2978),(2981,2954),(2954,2955),(2955,7067),(7067,2956),(2956,2965),(2965,2967),(2970,4988),(4988,2853),(2861,2829),(2829,2830),(2830,2831),(2831,2832),(2832,4349),(4349,3184),(3184,449),(453,4477),(4477,3321),(3321,4727),(4727,4728),(4728,461),(461,2068),(2068,2069),(2069,2070),(2070,2084),(2084,768),(768,769),(769,770),(770,771),(771,772),(772,773),(773,774),(774,775),(775,776),(776,777),(777,906),(786,792),(792,7586),(7588,281),(281,261),(3283,3284),(3284,3310),(3310,3481),(3481,3482),(3482,3483),(3483,3484),(3484,3485),(3485,3486),(3486,3487),(3487,7415),(7415,7416),(7416,7325),(7325,3317),(3317,3318),(3318,3320),(3320,2064),(2064,2065),(2065,4727),(7587,303),(6004,3),(3,192),(796,56),(56,58),(58,4499),(4499,4934),(4934,4500),(4500,4481),(4481,4485),(4485,822),(822,823),(823,824),(824,825),(825,826),(826,827),(827,71),(71,72),(72,5144),(5144,7027),(7027,4538),(4538,7396),(7396,5145),(5145,276),(485,487),(487,488),(488,489),(489,490),(490,491),(491,492),(492,493),(400,279),(279,3),(284,4504),(4504,7028),(7028,5147),(5147,68),(68,30),(30,69),(69,828),(828,829),(829,830),(830,831),(831,75),(75,4501),(4501,4935),(4935,4502),(4502,4936),(4936,79),(819,263),(263,6071),(263,4725),(4725,4495),(4495,2809),(2809,408),(408,409),(409,410),(410,411),(411,412),(412,413),(413,414),(414,415),(4525,4526),(4526,1700),(1702,16),(16,73),(73,74),(74,4371),(4371,832),(832,833),(833,77),(82,83),(83,84),(84,51),(52,263),(263,271),(271,1278),(1278,1282),(1285,1286),(1286,1288),(1289,1360),(1360,1362),(1362,1363),(1363,1365),(1365,1365),(1365,1366),(1366,1367),(1367,7455),(7455,7438),(7438,1410),(1410,1411),(1411,1412),(1412,1413),(1413,1414),(1414,1415),(1415,1417),(1417,1418),(1418,5149),(5149,1419),(1419,6094),(1423,1424),(1424,1425),(1425,1426),(1426,1427),(1427,1428),(1428,1429),(1429,1430),(1430,1432),(1432,1433),(1433,1434),(1434,1435),(1435,7439),(7439,7454),(7454,1380),(1380,1381),(1381,1382),(1382,1383),(1383,1384),(1384,1385),(1385,1348),(1359,279),(3,12),(14,53),(53,54),(54,55),(55,56),(60,820),(820,821),(821,63),(63,64),(64,65),(65,66),(66,4498),(4498,1660),(1661,4523),(4523,4524),(4524,4322),(6057,2183),(2185,1423),(1426,1463),(1463,4399),(4399,4400),(4400,1464),(1464,1469),(1469,1465),(1465,1466),(1466,1467),(1467,1468),(1468,1377),(1377,1378),(1378,4514),(4514,4515),(4515,1995),(1359,274),(274,1505),(1505,4509),(4509,499),(499,500),(500,501),(501,1484),(1484,1481),(1481,7320),(7320,1486),(1486,1487),(1487,1488),(1488,1489),(1489,1490),(1491,1492),(1492,1493),(1493,1482),(1482,1494),(1494,1495),(1495,1496),(1496,1515),(1515,1499),(1499,1485),(1485,4518),(4518,509),(509,510),(510,511),(511,513),(513,4513),(4513,272),(272,335),(335,1278),(1278,1934),(1934,1935),(1941,4516),(4516,1369),(1369,1370),(1370,1371),(1371,1454),(1454,1455),(1455,1453),(1453,1456),(1456,1457),(1457,1458),(1458,1459),(1459,1460),(1460,1462),(1462,5149),(6094,2102),(2103,7154),(7154,7153),(7153,6056),(7591,496),(523,1740),(1740,1741),(1741,1742),(1742,1743),(1743,1744),(1744,1745),(1745,1746),(1746,1747),(1747,1748),(1748,1749),(1749,1751),(1751,1752),(1752,1753),(1753,1754),(1754,1759),(1759,1758),(1758,1750),(1750,1769),(1769,1770),(1770,1771),(1771,1760),(1760,1761),(1761,1762),(1762,1763),(1763,1764),(1764,1765),(1765,1767),(1772,1773),(1773,1774),(1774,1775),(1775,1776),(1776,1777),(1777,1778),(1778,1755),(1755,1756),(1756,1757),(1757,1794),(1794,7469),(7469,1724),(1724,1725),(1725,1726),(1726,1727),(1727,1728),(1728,1729),(1729,1730),(1730,1731),(1731,1732),(1732,1733),(1733,1734),(1734,1735),(1735,1736),(1736,1737),(1737,1738),(1738,1739),(1739,614),(620,621),(621,293),(7149,7277),(7277,6363),(6363,6361),(6361,6367),(6367,4985),(4985,6219),(6219,6229),(6229,6364),(6364,1558),(1558,4542),(4542,1531),(1531,1532),(1532,1533),(1533,1534),(1534,1535),(1535,1536),(1536,1537),(1537,186),(186,187),(187,188),(188,189),(189,190),(190,191),(191,819),(819,264),(264,6059),(6059,334),(334,7581),(1076,1021),(1021,6042),(6042,1022),(1022,1023),(1023,1024),(1024,895),(896,897),(897,893),(893,894),(894,1067),(1067,1068),(1068,1069),(7582,319),(319,274),(274,7),(7,194),(194,195),(195,196),(196,197),(197,198),(198,199),(199,1506),(1506,1507),(1507,1508),(1508,1509),(1509,1510),(1510,1511),(1511,1512),(1512,1538),(1538,4543),(4543,1560),(1560,1280),(1280,6366),(6366,6230),(6230,6228),(6228,7111),(7111,6365),(6365,6276),(6276,6362),(6362,7276),(7276,7149),(319,7491),(3605,3606),(3606,3607),(3607,6051),(6051,6053),(6053,7327),(7327,3629),(3629,3630),(3630,3631),(3631,3632),(3632,3633),(3633,3634),(3634,3635),(3635,3636),(3636,905),(905,4331),(4331,4339),(4339,7692),(7692,6010),(6010,3656),(3656,7194),(7194,7196),(7196,6310),(6310,6311),(6311,7399),(7399,7398),(7398,7397),(7397,6252),(6252,301),(2809,2810),(2810,843),(843,1013),(1013,1015),(1015,1016),(895,1025),(1025,1026),(1026,1027),(1027,1028),(1028,2818),(2818,2819),(2819,3012),(3012,877),(877,876),(876,3011),(3011,2896),(2896,2897),(2897,1062),(1062,1063),(1063,1064),(1064,1065),(1065,1066),(1066,1067),(7582,407),(407,2498),(2498,2499),(2499,2500),(2500,2501),(2501,7623),(7623,6312),(6312,7197),(7197,7195),(7195,3653),(3653,6055),(6055,1073),(1073,3641),(3641,3642),(3642,3643),(3643,3583),(3583,3584),(3584,3585),(3585,3625),(3625,3626),(3626,3627),(3627,3628),(3628,6052),(6052,3613),(3613,3615),(3615,3616),(4320,1476),(1479,7622),(7622,334),(334,406),(406,747),(435,7353),(7353,4571),(4571,2013),(2013,2014),(2014,4636),(4636,2016),(2016,2015),(2015,2017),(2017,3259),(3259,7361),(7361,7362),(7362,3129),(3129,3130),(3130,3131),(3131,3132),(3132,3133),(3133,3134),(3134,3135),(3135,3136),(3136,3138),(3138,3139),(3139,4124),(4124,4125),(4125,4126),(4126,4127),(4127,4128),(4128,4129),(4129,4130),(4130,4131),(4131,7294),(7294,4132),(4132,4171),(4171,4172),(4172,4173),(4173,2962),(2962,2963),(2963,2964),(2964,4175),(4175,5091),(5091,5092),(5092,7574),(7574,4177),(4177,4178),(4178,4179),(4179,2993),(2993,2994),(2994,4180),(4180,4181),(4181,4182),(4182,4151),(4151,2997),(2997,4153),(4153,4154),(4154,4416),(4416,4201),(4201,4202),(4202,4203),(4203,5090),(5090,4204),(4204,4205),(4205,4206),(4206,3140),(3140,3141),(3141,3142),(3142,3143),(3143,3144),(3144,3145),(3145,3146),(3146,3147),(3147,3148),(3148,5127),(5127,5128),(5128,2996),(2996,3258),(3258,2060),(2060,2061),(2061,2062),(2062,2063),(2063,2064),(7588,325),(325,1443),(1445,4407),(4407,4320),(4533,4844),(4844,4177),(4495,1279),(1279,1934),(2311,5025),(5025,4854),(4854,1438),(1438,4855),(4855,1439),(1439,1440),(1440,1386),(1386,1387),(1387,1388),(1388,1389),(1389,3355),(3355,3356),(3356,1441),(1441,1442),(1442,7414),(7414,1396),(1396,2317),(2317,2318),(2318,2319),(2319,2320),(2320,2321),(2321,2322),(2322,6163),(6163,1095),(1095,1096),(1096,1097),(1097,1099),(1099,6132),(6132,4861),(4861,4862),(4863,4864),(4864,4865),(4865,4577),(4577,4578),(4578,1100),(1100,100),(100,1111),(1111,2323),(2323,2324),(2324,2325),(2325,2326),(2326,5150),(5150,2327),(2331,1397),(1397,1398),(1398,1399),(1399,5148),(5148,1401),(1401,1402),(1402,1403),(1403,1404),(1404,1405),(1405,4856),(4856,4857),(4857,307),(307,308),(308,309),(309,7412),(7412,7413),(7413,2383),(2383,2003),(2003,1358),(1358,4494),(309,2383),(7514,2279),(2279,6251),(6251,5140),(5140,1500),(2102,2726),(2726,2727),(2727,2105),(2105,6140),(6140,6144),(6144,6145),(6145,6146),(6146,6147),(6147,6149),(6149,6150),(6150,6143),(6143,6011),(6011,2133),(2133,6244),(6244,2134),(2134,2135),(2135,2164),(2164,2165),(4554,7016),(7016,4619),(2166,4554),(4606,7142),(7142,7015),(7015,4617),(3419,4762),(4762,4763),(4763,6245),(6245,2156),(2156,6014),(6014,6242),(6242,6152),(6152,6153),(6153,6243),(6243,6154),(6154,6155),(6155,6141),(6141,6142),(6142,2181),(2181,2182),(2182,2778),(2778,2185),(2383,1479),(7622,297),(297,407),(7623,2508),(2508,2278),(7698,7113),(7113,127),(127,112),(40,146),(146,147),(147,184),(184,185),(185,186),(819,4725),(4725,334),(7294,4156),(4156,4168),(4168,4170),(4170,4153),(792,320),(320,281),(281,6),(6,194),(199,200),(200,201),(201,148),(148,149),(149,150),(150,27),(94,126),(126,6182),(322,7697),(400,7581),(1085,1086),(1086,1087),(1087,1088),(1088,1089),(1089,1090),(1090,1091),(1091,1092),(1092,1093),(1093,1094),(1094,1095),(1096,1101),(1101,1102),(1102,1103),(1103,1104),(1105,1107),(1107,1108),(1108,1109),(1109,1110),(1110,1111),(1111,1112),(1112,1113),(1113,1114),(1114,1115),(1115,2437),(2437,1117),(1117,1118),(1118,1119),(1119,1120),(1120,1164),(7582,340),(2914,2915),(2915,2916),(2916,2917),(2917,2918),(2918,1300),(1303,5129),(5129,5130),(5130,1135),(1135,7064),(7064,7066),(7066,2924),(2924,2925),(2925,7442),(7442,7446),(7446,7070),(7070,7445),(7445,6339),(6339,6324),(6324,6325),(6325,6341),(6341,6277),(6277,6278),(6278,6279),(6279,6280),(6285,6286),(6286,6287),(6287,6288),(6288,6342),(6342,6298),(6298,6336),(6336,7069),(7069,7443),(7443,7444),(7444,2941),(2941,2942),(2942,7065),(7065,7063),(7063,1137),(1137,5131),(5131,5132),(5132,1331),(1334,2985),(2985,2986),(2986,2988),(2988,3001),(2925,2948),(2948,2949),(2949,2950),(2950,2954),(2965,2983),(2983,2984),(2984,2941),(1334,1335),(4407,2637),(2637,2721),(2721,7012),(7012,2201),(4599,4601),(4601,5058),(5058,4602),(4602,4603),(4603,4604),(4604,4605),(4605,4562),(4562,7386),(7386,4607),(4607,4608),(4608,4609),(4609,4610),(4610,4611),(4611,4612),(4612,4613),(4613,3370),(3370,7168),(7168,4616),(4620,7563),(7563,3368),(3368,4626),(4626,4627),(4627,4624),(4624,4625),(4625,4628),(4628,4623),(4623,4619),(4619,4561),(4561,4629),(4629,4630),(4630,4631),(4631,4632),(4632,5164),(5164,4633),(4633,2234),(2242,7435),(7435,2722),(2722,4413),(4413,1476),(4607,7140),(7140,7141),(7141,7138),(7138,3370),(3370,7231),(7231,7128),(7128,7117),(7117,7118),(7118,6289),(6290,7120),(7120,7119),(7119,7127),(7127,7143),(7143,3368),(3368,7137),(7137,7139),(7139,4623),(4599,7184),(7184,2225),(2225,4620),(7563,7231),(7143,7168),(4616,2227),(2227,7049),(7049,2236),(4953,768),(777,779),(779,780),(780,781),(781,782),(782,783),(786,791),(791,7585),(7585,7587),(7587,345),(345,318),(318,312),(2201,4401),(2219,3365),(3365,3366),(3366,3367),(3367,3886),(3886,3888),(3891,7187),(7187,4001),(4001,3949),(3950,3372),(3372,3373),(3373,3374),(3374,2233),(2235,2239),(315,4495),(2810,749),(753,754),(754,755),(755,756),(763,4953),(300,496),(1198,4782),(4782,1782),(1782,1783),(1783,1784),(1784,1785),(1785,1786),(1786,624),(624,625),(625,626),(626,627),(627,628),(628,629),(629,630),(630,4466),(4466,633),(633,634),(634,1005),(1005,1788),(1788,1789),(1789,7199),(956,957),(957,958),(958,959),(959,960),(960,961),(961,962),(962,635),(635,636),(636,637),(637,638),(638,639),(639,640),(640,641),(641,642),(642,643),(643,644),(644,645),(645,1796),(1796,1797),(1797,1798),(1798,1799),(1799,1800),(1800,1801),(1801,1277),(620,7568),(1791,1792),(1792,1793),(1793,962),(300,497),(1198,1222),(1222,1223),(1223,1224),(1224,1225),(1225,1226),(1226,1227),(1227,1228),(1228,1244),(1244,254),(254,1246),(1246,1229),(1229,1231),(1231,1232),(1232,1233),(1233,1234),(1234,7358),(7358,5174),(5174,6078),(6078,7112),(7112,972),(972,1625),(1629,6234),(1244,1246),(1227,1253),(1253,969),(969,1176),(1176,1177),(1177,1178),(1178,1179),(1179,7112),(324,1634),(1638,992),(992,7044),(7044,993),(993,5060),(5060,5061),(5061,1231),(7358,1235),(1235,1251),(1251,253),(253,5033),(5033,7130),(7130,1237),(1237,1238),(1238,1239),(1239,247),(247,1240),(1240,1241),(1241,1242),(1242,1277),(675,303),(1251,7130),(7358,1231),(1231,1235),(1179,5060),(4952,768),(768,2085),(2085,2086),(2086,479),(479,480),(480,481),(481,482),(482,483),(483,485),(485,488),(488,491),(491,493),(495,496),(496,523),(523,4382),(4593,4594),(664,614),(614,4415),(4415,302),(409,411),(411,414),(414,416),(416,417),(417,418),(418,419),(419,420),(420,2088),(2088,2089),(2089,763),(763,4952),(532,649),(649,650),(650,651),(651,4472),(4472,4386),(4386,653),(653,722),(722,723),(723,724),(724,725),(725,726),(726,727),(727,728),(728,729),(729,730),(730,5160),(5160,5155),(5155,4806),(4806,5156),(5156,7552),(7552,924),(924,925),(925,927),(927,938),(950,915),(915,916),(916,917),(917,918),(918,919),(919,7551),(7551,4800),(4800,5157),(5157,5154),(5154,4804),(4804,738),(738,739),(739,740),(740,741),(741,742),(742,743),(743,744),(744,745),(745,600),(600,654),(654,655),(655,656),(656,657),(657,658),(658,659),(659,604),(621,291),(720,680),(680,681),(681,682),(682,683),(683,684),(684,685),(685,686),(686,688),(688,689),(689,690),(690,5062),(5062,691),(691,692),(692,695),(695,696),(696,697),(697,698),(698,699),(699,700),(700,701),(701,702),(702,703),(703,704),(704,705),(705,706),(706,567),(549,677),(677,678),(678,679),(679,680),(692,694),(694,695),(706,707),(721,680),(692,693),(3640,3574),(3574,3641),(3585,3587),(3588,3589),(3589,3605),(3607,3608),(3608,3609),(3609,3598),(928,588),(588,590),(590,1729),(1729,1731),(1731,1738),(1738,614),(614,675),(675,621),(621,4495),(2810,842),(842,846),(846,848),(848,759),(768,770),(770,773),(774,906),(906,909),(909,790),(790,7585),(7587,496),(496,522),(522,1741),(1741,1747),(1747,1749),(1749,544),(544,546),(546,914),(3599,3610),(3610,3611),(3611,6135),(6135,3612),(3612,3613),(3619,6087),(6087,3620),(3620,3621),(3621,3622),(4387,3634),(4339,7616),(7616,5081),(5081,5082),(7623,3719),(3719,3718),(3718,3720),(3720,7691),(7691,3721),(3721,3722),(3722,5069),(5069,3723),(3723,3724),(3724,3725),(3725,3726),(3726,7039),(7039,7204),(7204,3728),(3728,3729),(3729,3730),(3730,3732),(3732,3733),(3733,3734),(3734,3735),(3735,7039),(7204,3736),(3736,3738),(3738,7170),(7170,3740),(3740,3741),(3741,5165),(5165,3743),(3743,3780),(3780,3744),(3744,3745),(3745,3746),(3746,3749),(3750,6252),(6252,302),(302,333),(333,747),(3714,3719),(3725,3738),(5165,7169),(7169,3743),(3744,3715),(3715,3755),(3773,5176),(5176,3774),(3798,6076),(7623,3715),(6050,7398),(7398,6252),(1808,1813),(4743,4745),(1841,1842),(1842,7374),(7374,1845),(1808,1816),(7343,7318),(7318,4980),(2338,1845),(1808,4744),(4989,1845),(324,327),(327,7113),(4725,273),(273,405),(405,408),(415,416),(420,421),(421,422),(422,423),(423,424),(424,425),(425,426),(426,427),(427,428),(428,429),(429,6334),(6334,7660),(7660,3114),(3114,3115),(3115,3116),(3116,6085),(6085,2026),(2026,7226),(7330,2051),(2051,6086),(6086,3161),(3161,3162),(3162,3163),(3163,3164),(3164,3084),(3084,469),(469,470),(470,471),(471,472),(472,473),(473,4705),(4705,475),(475,476),(476,477),(477,478),(478,479),(400,281),(323,332),(4962,4725),(429,3032),(3032,3033),(3033,3034),(3034,3036),(3036,3037),(3037,3038),(3038,3039),(3039,3040),(3040,3041),(3041,3042),(3042,2035),(2035,2036),(2036,2040),(2040,3047),(3047,3202),(3202,3203),(3203,3204),(3204,7641),(7641,7642),(7642,7643),(7643,3214),(3214,3215),(3215,3216),(3216,3217),(3217,3218),(3218,3219),(3219,3220),(3220,5046),(5046,7639),(1174,4725),(7639,7640),(7640,5047),(5047,3224),(3224,3225),(3225,3226),(3226,3227),(3227,3228),(3228,3229),(3229,7644),(7644,7645),(7645,7646),(7646,3238),(3238,3239),(3239,3240),(3240,3068),(3068,3069),(3069,3070),(3070,2041),(2041,3071),(3071,3072),(3072,3073),(3073,3074),(3074,3075),(3075,3076),(3076,3077),(3077,3084),(281,4),(4,4962),(7132,6230),(6228,4788),(4788,6273),(6273,136),(136,137),(137,193),(193,138),(138,167),(167,168),(168,169),(169,141),(141,142),(142,143),(143,115),(263,277),(277,336),(336,1278),(1291,2389),(2389,2390),(2390,2391),(2391,2392),(2392,2393),(2393,2394),(2394,1090),(1090,2413),(2413,2414),(2414,2415),(2415,2416),(2416,2417),(2417,2418),(2418,2419),(2419,2452),(2452,2453),(2453,2454),(2454,2455),(2455,2456),(2456,2445),(2445,2457),(2457,4392),(4392,2458),(2458,2446),(2446,2459),(2459,2460),(2460,2461),(2461,2462),(2462,2463),(2463,2431),(2433,2434),(2434,2435),(2435,2436),(2436,2437),(2437,2438),(2438,2439),(2439,2440),(2440,2441),(2441,2442),(2442,2443),(2443,2444),(2444,1344),(4,194),(91,129),(129,130),(130,131),(131,164),(164,165),(165,166),(166,7458),(7458,4479),(4479,4376),(4376,4377),(4377,4890),(4890,4369),(4369,6219),(6229,7131),(7588,328),(328,1443),(1911,1913),(1913,1914),(1914,1805),(1805,1806),(1814,1852),(1852,1853),(1853,1854),(1854,1844),(1844,6172),(4362,7025),(7025,4464),(4464,1869),(1869,1870),(1870,1871),(1871,1872),(1872,1873),(1873,1874),(1874,1875),(1875,1876),(1876,1877),(1877,1878),(1878,1879),(1879,1899),(1899,1887),(1887,1888),(1888,1889),(1889,1890),(1890,7240),(7240,1891),(1891,7156),(7156,7020),(7020,7029),(7029,7038),(7038,7011),(7011,2171),(2171,7160),(7160,7700),(7700,7047),(7047,7161),(7161,7162),(767,768),(7158,7048),(7048,7699),(7699,7159),(7159,7388),(7388,7017),(7017,7018),(7018,7030),(7030,7021),(7021,7163),(7163,1893),(1893,1894),(1894,1912),(1912,1895),(1895,1896),(1896,1897),(1897,6088),(6088,1898),(1898,1858),(1858,1859),(1859,1860),(1860,4492),(4492,1861),(1861,1862),(1862,1863),(1863,1864),(1864,1865),(1865,1866),(1866,1850),(1850,1867),(1867,4489),(4489,4747),(4903,1883),(1883,1884),(1884,1902),(1902,1903),(1903,1904),(1904,1840),(1905,1906),(1906,1907),(1907,1908),(1908,1909),(1909,1713),(315,335),(335,404),(753,7339),(7132,1558),(1558,2957),(2957,2958),(2958,4938),(4938,1549),(1549,1523),(1523,1524),(1524,1525),(1525,1526),(1526,6080),(6080,1585),(1585,1586),(1586,1587),(1587,4539),(4539,1606),(1606,1607),(1607,1608),(1608,1609),(1609,1610),(1610,7471),(7471,1533),(186,838),(838,839),(839,840),(840,841),(841,49),(1947,2642),(2642,2643),(2643,2644),(2644,2709),(2709,2710),(2710,2711),(2711,2712),(2712,2655),(2655,2656),(2656,2668),(2668,2672),(2672,2673),(2673,2206),(2206,4798),(4798,2674),(2674,2675),(2675,2124),(2124,2125),(2125,2126),(2126,2127),(2127,2128),(2128,2129),(2129,4797),(4797,7473),(7473,2679),(2679,4691),(4691,2682),(2682,4796),(4796,4690),(4690,4411),(4411,4715),(1587,1591),(1591,4542),(190,192),(192,819),(819,261),(4795,4688),(4688,4689),(4689,4410),(4410,2678),(2678,2681),(2681,2680),(2680,2676),(2676,2118),(2118,2119),(2119,2120),(2120,2121),(2121,2123),(2123,2685),(2685,2686),(7510,4799),(4799,2205),(2205,2687),(2687,2688),(2688,2689),(2689,2696),(2696,2697),(2697,2713),(2713,2714),(2714,2715),(2715,2716),(2716,2718),(2718,2719),(2719,1989),(1358,319),(319,279),(279,7),(15,834),(834,835),(835,836),(836,837),(837,1506),(1511,1592),(1592,1593),(1593,1594),(1594,1595),(1595,1596),(1596,4363),(4363,1598),(1598,1597),(1597,1599),(1599,1573),(1573,1574),(1574,1575),(1575,4583),(4583,1518),(1518,1519),(1519,1520),(1520,7661),(7661,1521),(1521,4546),(4546,1559),(1559,987),(987,4937),(4937,1547),(1547,1548),(1548,1280),(1280,6348),(288,1171),(1630,7348),(7348,3669),(3669,3671),(3676,5073),(5073,5074),(5074,6054),(6054,4330),(4330,3679),(3679,5075),(5075,5076),(5076,3682),(3682,3864),(3864,3865),(3865,4910),(4910,4911),(4911,4912),(4912,4913),(4913,4914),(4914,4915),(4915,4957),(4843,4905),(4905,4906),(4906,4907),(4907,4908),(4908,4909),(4909,3884),(3884,3686),(3686,5077),(5077,5078),(5078,3689),(3689,3690),(3690,6117),(6117,5079),(5079,3694),(3885,3663),(3663,7347),(277,288),(7144,3881),(3881,5096),(5096,7145),(7145,4843),(1184,497),(6115,1205),(1205,1206),(1206,1207),(1207,1208),(1208,1209),(1209,1210),(1210,1211),(1211,3575),(3575,3576),(3576,3577),(3577,3578),(3578,3579),(3579,3580),(3580,3640),(5082,3639),(3639,3644),(3644,3645),(3645,3646),(3646,3647),(3647,3661),(3661,3573),(3573,3662),(3662,1212),(1212,1213),(1213,1214),(1214,1215),(1215,1216),(1216,1217),(1217,4563),(675,1184),(1210,3575),(3576,3648),(3648,3649),(3649,3650),(3650,7328),(7328,3651),(3651,3652),(3652,3653),(3653,3654),(3654,5084),(5084,6054),(3712,6359),(6359,7189),(5044,4433),(4433,3752),(6117,3655),(3655,5086),(5086,3656),(3656,3657),(3657,3658),(3658,7329),(7329,3659),(3659,3660),(3660,3647),(4923,4924),(4924,4958),(4958,3752),(3658,3659),(7571,207),(207,208),(350,2809),(2810,2811),(2811,843),(1015,2812),(2812,2813),(2813,2814),(2814,852),(855,2815),(2815,2816),(2816,2817),(2817,2818),(2819,2820),(2820,2822),(2822,2823),(2823,5032),(5032,2824),(2824,2825),(2825,2826),(2826,2827),(2827,2828),(2828,2829),(2832,2833),(2833,7375),(7375,3466),(3466,3467),(3467,3469),(3469,3470),(3470,3471),(3471,3472),(3472,3473),(3473,3474),(3474,3475),(3475,3476),(3476,3477),(3477,3478),(3478,3280),(3280,3282),(3282,3479),(3479,4086),(4086,4087),(4087,4088),(4088,4089),(4089,4090),(4090,4091),(4091,6097),(6097,4093),(4093,4094),(4094,4096),(4096,4114),(4114,4115),(4115,4116),(4116,4117),(4117,4118),(4118,4119),(4119,4120),(4120,4121),(4121,4122),(4122,4123),(4123,3283),(3487,3488),(3488,3489),(3489,3492),(3492,2835),(2835,7176),(7176,2836),(2836,2837),(2840,2841),(2841,2865),(2866,2892),(2892,2893),(2893,2894),(2894,2895),(2895,2896),(2897,2898),(2898,3562),(3562,2899),(2899,884),(887,2900),(2900,2901),(2901,2902),(2902,2903),(2903,1074),(1074,6074),(6074,793),(793,495),(205,206),(206,207),(210,1643),(1643,7571),(7564,340),(380,2087),(2087,7053),(7053,2088),(7057,5142),(5142,7567),(7567,7326),(7326,7418),(7418,7417),(7417,3471),(3473,5015),(5015,7290),(7290,5016),(2008,2071),(2071,2072),(2072,2073),(2073,2074),(2074,2075),(2075,2076),(2076,460),(460,2011),(5013,5014),(5014,7291),(7291,3485),(7325,4984),(4984,448),(448,3181),(3321,2066),(2066,2067),(2067,4728),(2086,4455),(4455,381),(400,7564),(7564,7663),(4728,437),(437,459),(459,2080),(2080,2081),(2081,2082),(2082,2083),(2083,2084),(342,4521),(4521,1934),(1934,2385),(2385,2386),(2386,2387),(2387,2388),(2388,1290),(1298,1121),(1140,2515),(2515,2516),(2516,2517),(2517,2518),(2518,4749),(4749,4750),(4750,4751),(4751,4754),(4754,4755),(4755,3003),(3003,2522),(2522,2524),(2524,2523),(2523,2532),(2532,4877),(4877,4878),(4878,7180),(7180,7123),(7123,4347),(4347,7181),(4342,4348),(4348,7122),(7122,7121),(7121,7183),(7183,4880),(4880,2539),(2539,2540),(2540,2542),(2542,3004),(3004,4756),(4756,3005),(3005,4757),(4757,4758),(4758,6343),(6343,4759),(4759,4760),(4760,2547),(2547,2548),(2548,946),(1163,1337),(1347,2634),(2634,2635),(2635,2636),(2636,5097),(5097,2383),(1358,5192),(5192,342),(2353,2357),(2357,2359),(2359,7122),(4380,496),(4384,518),(518,2255),(2255,2256),(2256,2717),(2717,2257),(2257,2258),(2258,2259),(2259,2260),(2260,2261),(2261,4469),(4469,2262),(2262,2264),(2264,2265),(2265,2269),(2269,2270),(4380,1172),(1172,7671),(7671,513),(4513,286),(286,271),(271,496),(2270,2274),(2274,7601),(7601,2508),(2278,2279),(2279,2720),(2720,2280),(2280,2281),(2281,2282),(2282,4468),(4468,2283),(2283,616),(675,4380),(302,340),(5032,2867),(2867,1035),(1035,1314),(1314,1316),(1316,1315),(1315,1317),(1317,1318),(1318,1319),(1319,1281),(1281,1329),(1330,2919),(2919,2920),(2920,2921),(2921,2922),(2922,2923),(2923,2924),(2954,7067),(7067,2955),(2955,2984),(2984,2927),(2927,2929),(2929,2930),(2930,7441),(7441,2933),(2984,7449),(7449,2941),(2942,2943),(2943,2944),(2944,2945),(2945,2946),(2946,2947),(2947,1305),(1305,1306),(1306,1307),(1307,1308),(1308,1309),(1309,1310),(1310,1311),(1311,1312),(1312,1313),(1313,1055),(1055,2868),(2868,2892),(400,303),(2934,2935),(2935,2936),(2936,2937),(2937,2938),(2938,7440),(7440,2940),(2940,2948),(7564,4521),(4521,1283),(1283,4456),(1130,2550),(2550,2551),(2551,2553),(2553,2554),(2554,2555),(2555,2556),(2556,2557),(2351,4445),(4445,2352),(2352,2573),(2573,2574),(2574,2577),(2577,2578),(2578,2579),(2579,7395),(7395,6124),(6124,4004),(4004,4005),(4005,4006),(4006,4007),(4007,4008),(4008,4010),(4010,4009),(4009,4011),(4011,5110),(5110,4012),(4012,4013),(4013,4014),(4014,4098),(4098,4015),(4015,4017),(4017,4018),(4018,4019),(4019,4020),(4020,4100),(4100,4021),(4021,4022),(4022,4023),(4023,4024),(4024,4027),(4027,7250),(7250,7248),(7248,7207),(7207,7288),(7288,7286),(7286,7283),(7283,7282),(7282,7278),(7278,7289),(7250,7252),(7252,7248),(7248,7254),(7254,7257),(7257,7258),(7258,7260),(7260,7263),(7263,7264),(7264,7268),(7268,7270),(5111,4051),(4051,4052),(4052,4054),(4054,4055),(4055,4057),(4057,4101),(4101,4102),(4102,4058),(4058,4059),(4059,4060),(4060,4061),(4061,4062),(4062,4063),(4063,4064),(4064,4056),(4056,4133),(4133,4065),(4065,4066),(4066,4067),(4067,4068),(4068,4069),(4069,4070),(4070,4071),(4071,4072),(4072,4073),(4073,2583),(2583,2584),(2584,2585),(2585,2586),(2586,2587),(2587,2588),(2588,2589),(2589,2590),(2590,2591),(2591,2592),(2592,2357),(2357,2358),(2358,2359),(4435,2594),(2594,2595),(2595,2596),(2596,2597),(2597,2599),(2599,2600),(2600,1155),(1072,1352),(1357,1358),(1358,7564),(7289,7280),(7280,7281),(7281,7284),(7284,7287),(7287,7208),(7208,7249),(7249,7253),(7253,7251),(7251,4051),(7270,7266),(7266,7265),(7265,7262),(7262,7261),(7261,7259),(7259,7256),(7256,7255),(7255,7249),(3888,7186),(7186,7187),(7187,3954),(3954,3955),(3955,3956),(3956,3957),(3957,3958),(3958,3959),(3959,3960),(3960,3961),(3961,3962),(3962,3963),(3963,3964),(3964,3965),(3965,3968),(3968,7771),(7771,3966),(3966,5113),(5113,3918),(3918,5114),(3980,6089),(6089,3981),(3981,3982),(3982,7704),(7704,3983),(3983,3985),(3985,3986),(3986,3987),(3987,3989),(3989,3990),(3990,3991),(3991,3998),(3998,3999),(3999,4000),(4000,4001),(3986,7313),(7313,6164),(6164,6159),(6159,6160),(6160,6161),(6161,3991),(7187,3947),(3947,3892),(3892,3893),(3893,3895),(3895,3896),(3896,3897),(3897,3898),(3898,3899),(3899,3900),(3900,3901),(3901,3902),(3902,3904),(3904,3905),(3905,3906),(3906,3907),(3907,3908),(3908,3909),(3909,5118),(5118,3910),(3910,3911),(3911,3912),(3912,3913),(3913,3914),(3914,7075),(7075,3925),(3925,3008),(3008,6089),(3917,3918),(3918,3926),(3926,7074),(7074,3919),(3919,3920),(3920,3921),(3921,3922),(3922,3923),(3923,3112),(3112,3924),(3924,3927),(3927,3928),(3928,3929),(3929,7108),(7108,3935),(3935,3936),(3936,7110),(7110,3938),(3938,3939),(3939,3940),(3940,3941),(3941,3943),(3943,3944),(3944,3945),(3945,3946),(3946,4001),(2810,7385),(4495,1278),(1367,1369),(1371,1372),(1372,1373),(1373,1374),(1374,1946),(4379,1968),(1968,1969),(1969,1970),(1970,1971),(1971,2114),(2114,2115),(2115,6011),(2134,3378),(3378,3379),(3379,4669),(4669,2165),(2166,2167),(2167,3383),(3383,3384),(3384,3385),(3385,3386),(3386,3387),(3387,4559),(4559,3389),(3389,3390),(3390,6119),(6119,3392),(3392,3393),(3393,3394),(3394,3396),(3396,3397),(3397,3398),(3398,3399),(3399,6187),(6187,6188),(6188,6215),(6215,6184),(3400,3401),(3401,3402),(3402,3403),(3403,3404),(3404,3405),(3405,3406),(3406,3410),(3410,3407),(3407,3408),(3408,3409),(3409,4558),(4558,3411),(3411,3412),(3412,3413),(3413,3414),(3414,3415),(3415,3416),(3416,3417),(3417,3418),(3419,3420),(3420,3421),(3421,3422),(3422,3423),(3423,2156),(6014,2158),(2158,2159),(4409,1972),(1972,1973),(1973,1974),(1974,1975),(1989,1375),(1375,1376),(1376,1377),(1378,1379),(1379,1380),(1358,7665),(1989,1472),(6185,6186),(6186,6216),(6216,3400),(4409,2173),(2173,2174),(2174,3425),(3425,2176),(2176,2177),(2177,2178),(2178,7354),(7354,1975),(4495,4720),(4720,1443),(1451,1452),(1452,2640),(2640,2641),(2641,1948),(1971,2160),(4667,2162),(2162,2163),(2163,2168),(2168,4536),(4536,6003),(6003,5123),(5123,5124),(5124,5125),(5125,6001),(6001,4438),(4438,3432),(3432,3426),(3426,3434),(3434,3435),(3435,3453),(3453,3437),(3437,3439),(3439,3440),(3440,4557),(4557,3441),(3441,4560),(4560,5126),(4564,3443),(3443,3444),(3444,3445),(3445,3446),(3446,3447),(3447,3448),(3448,3449),(3449,3436),(3436,7432),(7432,3454),(3454,3455),(3455,3456),(3456,3457),(3457,3438),(3438,5022),(5022,6002),(6002,5119),(5119,5120),(5120,5121),(5121,5122),(5122,3461),(3461,4532),(4532,4534),(4534,4535),(4535,2169),(2169,2170),(2170,4672),(1981,4406),(1988,2642),(2642,7435),(2722,1471),(1471,1472),(7453,1480),(1480,2912),(2912,1358),(1808,7389),(7389,4324),(4324,4769),(4769,4770),(4770,4765),(4765,4766),(4766,4767),(4767,4768),(4768,4325),(4325,3328),(3328,3329),(3329,3330),(3330,3331),(3331,3351),(3351,3332),(3332,3333),(3333,3334),(3334,3352),(3352,3335),(3335,3336),(3336,3337),(3337,3338),(3338,3339),(3339,3340),(3340,4324),(4768,1845),(326,1443),(2637,2638),(2638,2640),(2641,2643),(2644,2645),(2645,2646),(2646,2647),(2647,2648),(2648,2649),(2649,2650),(2650,2652),(2652,2653),(2653,2654),(2654,2655),(2656,2657),(2657,2658),(2658,2659),(2659,2660),(2660,2661),(2661,2662),(2662,2909),(2909,2906),(2906,2907),(2907,2908),(2670,2910),(2910,2911),(2911,7507),(7507,2663),(2663,2664),(2664,2665),(2665,2666),(2666,2667),(2667,2696),(2697,2700),(2700,2701),(2701,2702),(2702,2703),(2703,2704),(2704,2706),(2706,6070),(6070,2708),(2708,5134),(5134,2718),(2719,7435),(1358,326),(324,1555),(1555,1556),(1556,1557),(1557,1181),(1181,7350),(7350,6230),(6273,4891),(4891,7458),(7458,7425),(7425,7424),(7424,7423),(7423,1566),(1566,1567),(1567,4372),(4372,179),(179,7431),(7431,175),(175,176),(176,181),(181,182),(182,183),(183,184),(190,1614),(1614,1615),(1615,1478),(315,336),(1279,1283),(1077,2479),(2479,2480),(2480,2481),(2481,2482),(2482,2483),(2483,2484),(2484,2485),(2485,2486),(2486,2487),(2487,2488),(2488,2489),(2489,2448),(2448,5026),(5026,2450),(2450,2490),(2490,2491),(2491,2492),(2492,2493),(2493,2465),(2465,2466),(2466,2467),(2467,2494),(2494,2495),(2495,2496),(2496,2497),(2497,2471),(2471,2472),(2472,2473),(2473,2474),(2474,2475),(2475,2476),(2476,2477),(2477,2478),(2478,1170),(319,1443),(1443,1616),(1616,1617),(1617,1618),(1618,1619),(1619,195),(201,202),(202,151),(151,153),(153,154),(154,155),(155,156),(156,7430),(7430,177),(177,178),(178,1539),(1539,1540),(1540,7426),(7426,7427),(7427,7428),(7428,138),(138,133),(133,134),(134,4890),(6229,6348),(6348,1182),(1182,1550),(1550,1551),(1551,1552),(1552,332),(7188,3082),(3082,7660),(3116,3117),(3117,3118),(3118,3119),(3119,3120),(3120,3122),(3122,3123),(3123,3124),(3124,3125),(3125,3126),(3126,3127),(3127,3128),(3128,3130),(3133,5046),(7640,3134),(4170,4132),(4132,4134),(4134,4135),(4135,4136),(4136,4207),(4207,4208),(4208,4209),(4209,4210),(4210,4211),(4211,4212),(4212,4213),(4213,5139),(5139,4214),(4214,4215),(4215,4216),(4216,4217),(4217,4218),(4218,4219),(4219,4220),(4220,4221),(4221,4222),(4222,4224),(4224,4225),(4225,4226),(4226,4227),(4227,4228),(4228,4229),(4229,4230),(4230,4231),(4231,7352),(7352,4246),(4246,4248),(4248,4249),(4249,4250),(4250,4251),(4251,4252),(4252,4253),(4253,4254),(4254,6137),(6137,4256),(4256,4257),(4257,4258),(4258,4259),(4259,7272),(7272,4956),(4253,4305),(4305,4306),(4306,4309),(4309,4254),(4260,4261),(4261,7271),(7271,5136),(5136,4262),(4262,4263),(4263,4264),(4264,7296),(7296,4265),(4265,4266),(4266,4267),(4267,4268),(4268,4269),(4269,4270),(4270,4271),(4271,4273),(4273,4274),(4274,4275),(4275,4276),(4276,4277),(4277,4278),(4278,4279),(4279,4280),(4280,4281),(4281,4282),(4282,4283),(4283,4284),(4284,4285),(4285,4286),(4286,4287),(4287,4288),(4288,4295),(4295,4296),(4296,4297),(4297,4299),(4299,4300),(4300,4301),(4301,4302),(4302,5135),(5135,4303),(4303,4304),(4304,4148),(4148,4149),(4149,3045),(3045,4150),(4150,4151),(4151,4156),(4203,4204),(3144,5047),(5047,3146),(3148,3149),(3149,3150),(3150,3151),(3151,3153),(3153,3152),(3152,2995),(2995,3154),(3154,3155),(3155,3156),(3156,3158),(3158,3159),(3159,3160),(3160,3161),(3164,3079),(3079,3080),(3080,3081),(3081,7188),(4266,4307),(4307,4308),(4308,4267),(4319,1476),(7453,2001),(1359,4724),(4724,263),(263,4508),(4508,7402),(7402,4717),(4717,2499),(2499,7216),(7216,2501),(7623,7401),(7401,3665),(3665,7400),(7400,3669),(3669,7399),(7399,7611),(7611,7398),(7397,5140),(5140,620),(620,1172),(1172,286),(271,4521),(1935,1445),(1445,4319),(6074,748),(749,494),(495,7588),(7588,4717),(3669,7611),(2809,7612),(7612,2811),(2811,842),(842,1014),(1014,1015),(1015,1074),(7389,7025),(1899,6107),(6107,6108),(6108,6109),(6109,6110),(6110,7020),(7021,6111),(6111,6112),(6112,6113),(6113,1898),(4747,1845),(763,767),(783,785),(7587,328),(1443,1647),(1647,1649),(1911,1805),(1805,1662),(1662,1666),(1666,1808),(1808,7475),(7475,6107),(6107,6109),(6110,7029),(7011,7160),(7159,7017),(7030,6111),(6113,4323),(4323,1845),(1845,1696),(1696,1700),(1700,1906),(1906,1909),(1713,1715),(1715,1478),(1478,315),(749,751),(752,7339),(4592,980),(980,981),(981,1518),(1559,1560),(1552,1553),(1553,7023),(7023,7014),(7014,4315),(4315,4316),(6004,7),(1538,4548),(4548,980),(4316,4317),(4317,7013),(7013,6162),(6162,7022),(7022,1554),(1554,1555),(7350,1558),(1558,1549),(6080,983),(983,984),(984,985),(985,1529),(1529,4542),(52,4725),(4725,6071),(4548,1514),(1514,980),(981,1576),(1576,1577),(1577,1578),(1578,4364),(4364,1579),(1579,1580),(1580,7237),(7237,7241),(7241,7247),(7247,7243),(7243,7126),(7318,4978),(4978,4979),(4979,4986),(4986,7107),(7107,7102),(7102,7103),(7103,7104),(7104,7105),(7105,7106),(7106,6005),(6005,1546),(1546,7072),(7343,4978),(1546,7560),(7073,7097),(7097,7098),(7098,7099),(7099,7100),(7100,7101),(7101,7505),(7505,5071),(5071,5072),(5072,4321),(4321,7342),(7344,2510),(2510,7242),(7242,7238),(7238,7224),(7224,1581),(1581,1582),(1582,4793),(4793,6068),(6068,1584),(1584,983),(5072,7342),(7559,7073),(7672,826),(5144,7702),(7702,7009),(7009,1604),(1604,1573),(4583,1576),(1579,7673),(7673,7674),(7674,7675),(7675,7676),(7676,7677),(7677,7559),(7559,7107),(6005,7072),(7505,7678),(7678,7560),(7560,7679),(7679,7680),(7680,7681),(7681,7738),(7738,7683),(7683,1582),(1584,1585),(1586,1600),(1600,7701),(7701,7010),(7010,7703),(7703,5147),(69,7686),(3865,7133),(7133,7134),(7134,3867),(3867,7135),(7135,7136),(7136,3874),(7348,3671),(4957,7133),(7145,3884),(3690,5079),(3675,5030),(5030,5031),(5031,6033),(6033,3089),(3089,3090),(3090,3706),(3706,3707),(3707,3708),(3708,3709),(3709,3710),(3710,4925),(4925,4926),(4926,3678),(3712,4916),(4916,4917),(4917,4918),(4918,4919),(4919,4920),(4920,4910),(4909,4921),(4921,4922),(4922,4923),(3691,3572),(3572,3701),(3701,3702),(3702,3088),(3088,3704),(3704,3705),(3705,733),(733,6015),(6015,5029),(5029,5028),(5028,3696),(3711,7189),(5044,3751),(7587,407),(7623,3676),(5073,3679),(4915,7654),(7623,5030),(4925,3679),(7654,4843),(5077,3694),(3695,7115),(7115,6310),(6311,4495),(760,762),(3751,7114),(7114,7116),(7116,3694),(5028,7115),(2833,6006),(6006,3493),(3493,3494),(3494,3495),(3495,3496),(3496,3497),(3497,3498),(3498,3499),(3499,3500),(3500,3501),(3501,3502),(3502,3503),(3503,3504),(3504,3505),(3505,3506),(3506,3507),(3507,3508),(3508,3510),(3510,3512),(3512,3514),(3514,3515),(3515,3516),(3516,3517),(3517,3518),(3518,3519),(3519,3521),(3521,3523),(3523,3524),(3524,3525),(3525,3527),(3527,3528),(3528,3529),(3529,3532),(3532,3533),(3533,3491),(3491,6007),(6007,2836),(2866,2825),(807,808),(808,809),(809,810),(810,811),(811,812),(812,813),(813,814),(814,81),(81,817),(2017,2018),(2018,2019),(2019,2020),(2020,2021),(2021,2022),(2022,4565),(4565,4566),(4566,4567),(4567,2031),(2031,2032),(2032,2033),(2033,2034),(2034,2035),(2036,2039),(4962,264),(2039,2040),(2040,2041),(2041,2042),(2042,2043),(2043,2044),(2044,2045),(2045,2046),(2046,4568),(4568,4569),(4569,4570),(4570,2056),(2056,2057),(2057,2058),(2058,2059),(2059,2060),(278,2),(796,797),(797,798),(798,799),(799,800),(800,801),(801,802),(802,803),(803,804),(804,7513),(7513,807),(7491,278),(2,4962),(3085,3034),(3034,3114),(3114,4353),(4353,4355),(4355,4354),(4354,3196),(3196,3345),(3345,4727),(2,4726),(4726,7555),(7555,4962),(7392,1443),(2638,2721),(2213,4688),(2680,4677),(4670,3420),(3422,4671),(2172,4409),(2160,4665),(4668,3376),(3376,3377),(3377,3378),(4669,2146),(6245,6013),(6013,6247),(6247,2679),(4691,4796),(4411,2239),(2811,749),(7433,4677),(2137,2138),(3423,4409),(1975,1982),(1982,2642),(7435,4413),(4413,7453),(342,1279),(2394,2395),(2395,2396),(2396,2397),(2397,2398),(2398,2399),(2399,2400),(2400,2401),(2401,2402),(2402,2403),(2403,1096),(6132,2551),(4640,7214),(7214,4347),(2351,2558),(2558,6170),(6170,5023),(5023,5024),(5161,5162),(5162,2571),(2571,2359),(4647,4435),(2599,4577),(1111,2404),(2404,2405),(2405,2406),(2406,2408),(2408,2409),(2409,2410),(2410,2411),(2411,2412),(2412,2438),(2636,2383),(1358,342),(2105,2728),(2728,2729),(2729,2730),(2730,6083),(6083,2733),(2733,2734),(2734,5169),(5169,2735),(2735,2736),(2736,2725),(2725,2737),(2737,2738),(2738,2739),(2739,7177),(7177,2740),(2740,2741),(2741,2742),(2742,2743),(2743,2745),(2745,2746),(2746,2747),(2747,2748),(2748,7062),(2360,4340),(4344,2349),(2622,2750),(2750,2751),(2751,2752),(2752,2753),(2753,2754),(2754,2755),(2755,2756),(2756,2764),(2764,7178),(7178,2765),(2765,2766),(2766,2767),(2767,2772),(2772,2768),(2768,2769),(2769,2770),(2770,2771),(2771,2773),(2773,2774),(2774,2777),(2777,2775),(2775,2181),(1358,340),(2550,2516),(4750,3006),(3006,4871),(4871,4872),(4872,4873),(4873,4874),(4874,2532),(7180,7331),(7331,2559),(2559,2560),(2560,2561),(2561,2562),(2562,2564),(2564,4927),(4927,4928),(4928,2352),(2574,2575),(2575,4929),(4929,7460),(7460,4930),(4930,4931),(4931,7459),(4713,4714),(4714,4960),(4960,4932),(4932,4933),(4933,2590),(2357,2566),(2566,2567),(2567,2568),(2568,2569),(2569,7550),(7550,2570),(2570,7222),(7222,6170),(6170,7183),(2540,4882),(4882,4883),(4883,4884),(4884,4885),(4885,6343),(2548,2600),(3957,3993),(3993,3994),(3994,3995),(3995,3996),(3996,3997),(3997,3998),(3959,7234),(7234,4002),(4002,5112),(5112,4003),(4003,4404),(4404,4405),(4405,3991),(3960,6156),(6156,6165),(6165,6157),(6157,6158),(6158,7314),(7314,3964),(3968,3966),(3982,3983),(1445,1447),(1447,2637),(2215,7184),(7184,7050),(7050,7185),(7185,7186),(6158,3964),(4001,7193),(7193,7049),(2236,2239),(3996,3998),(3905,7085),(7085,7086),(7086,7087),(7087,7088),(7088,7089),(7089,7091),(7091,3910),(3914,3925),(3907,3909),(3923,7079),(7079,7080),(7080,7081),(7081,7565),(7565,7082),(7082,7083),(7083,7084),(7084,7108),(3936,3938),(1374,1375),(6155,1987),(1987,1989),(1989,1995),(1995,1998),(1998,2002),(2637,1956),(1956,2109),(2109,3427),(3427,3428),(3428,3429),(3429,3432),(3432,3434),(3434,3453),(3437,3440),(3440,3441),(4564,3444),(3444,3447),(3447,3454),(3455,5022),(5022,3461),(3461,4537),(4537,2176),(2176,1982),(1982,7435),(4413,1480),(1480,1358),(2339,2420),(2420,2421),(2421,2423),(2423,2424),(2424,2447),(2447,5133),(5133,2596),(2596,2611),(2611,2602),(2602,2613),(2613,2614),(2614,2615),(2615,2535),(2535,2536),(2536,2616),(2616,2617),(2617,2557),(2558,2559),(2570,2571),(4435,2604),(2604,2605),(2605,2538),(2538,2606),(2606,2607),(2607,2608),(2608,2609),(2609,2610),(2610,2554),(2554,2425),(2425,2426),(2426,2427),(2427,2428),(2428,2429),(2429,2430),(2430,2371),(2569,2570),(5024,5161),(5162,4880),(4880,2606),(2373,1424),(1424,2190),(5192,350),(2810,750),(7012,2644),(2909,6020),(6020,6023),(6023,6026),(6026,6027),(6027,6029),(6030,6028),(6028,6025),(6025,6024),(6024,6021),(6021,7507),(2718,7435),(3041,2035),(3204,3205),(3205,3206),(3206,3207),(3207,3208),(3208,3209),(3209,3210),(3210,4981),(4981,3211),(3211,3212),(3212,3213),(3213,3214),(3220,3221),(3221,4982),(3222,3223),(3223,3224),(3229,7472),(7472,3230),(3230,3232),(3232,3233),(3233,3234),(3234,4462),(4462,3235),(3235,3236),(3236,3237),(3237,3238),(3077,3079),(3081,3084),(4962,6059),(7353,7584),(7584,3344),(3344,7421),(7421,7422),(7422,6085),(6085,2053),(2053,2054),(2054,2055),(2055,4732),(4732,3243),(3243,3244),(3244,3245),(3245,3212),(3219,3537),(3537,3538),(3538,7556),(7556,3539),(3539,3540),(3540,3541),(3541,3542),(3542,3544),(3544,3552),(3552,3553),(3553,3554),(3554,3555),(3555,3556),(3556,3557),(3557,7317),(7317,3224),(3230,3231),(3231,3246),(3246,3247),(3247,7056),(7056,4731),(4731,2023),(2023,2024),(2024,2025),(2025,6086),(6086,7419),(7419,7420),(7420,3345),(345,320),(4725,6059),(2055,4565),(4567,3238),(3240,3048),(3048,3049),(3049,3050),(3050,3051),(3051,3052),(3052,3053),(3053,3054),(3054,3055),(3055,3056),(3056,3057),(3057,4830),(4830,7667),(7667,3091),(3091,3092),(3092,3093),(3093,3535),(3535,3536),(3536,3537),(3542,3543),(3543,3141),(3142,3224),(478,7053),(2089,770),(772,417),(417,485),(495,7587),(3057,3059),(3059,3060),(3060,3061),(3061,3062),(3062,3063),(3063,3064),(3064,3065),(3065,3066),(3066,3067),(3067,3202),(3204,4568),(4569,2023),(4,7555),(1567,173),(173,174),(174,175),(156,157),(157,158),(158,159),(159,1539),(3082,3114),(3144,7639),(3164,7188),(3080,3082),(3082,3084),(4495,746),(746,844),(844,846),(848,758),(758,760),(760,2007),(2007,4571),(4571,2014),(2014,2016),(2016,2017),(2017,7362),(7362,3134),(3135,7363),(7363,4588),(4588,4589),(4589,7364),(7364,7365),(7365,4210),(4211,4215),(4216,4222),(4225,7462),(7462,7461),(7461,4230),(4230,4248),(4248,4252),(4252,6137),(6137,4956),(342,4495),(4252,4305),(4306,4307),(4308,4253),(4253,4268),(4268,4273),(4273,4277),(4277,7462),(7461,4281),(4284,4295),(4296,4301),(4302,4591),(4591,4590),(4590,7366),(7366,7367),(7367,7368),(7368,3143),(3145,5128),(5128,2060),(2060,2062),(2062,2064),(2064,4727),(4727,768),(768,772),(772,775),(775,906),(909,792),(4260,7296),(7296,4266),(4266,4253)]

In [ ]:
# loop through each segment and check how many rows we have for it
# if there are no rows for a particular segment, then add it to the missing list
missing = []
for segment in segments:
    print(str(segment[0]) + " to " + str(segment[1]) + ": ")
    df_temp = df.loc[(df['stoppointid_first'] == segment[0]) & (df['stoppointid_next'] == segment[1])]
    print(str(df_temp.shape[0]) + "\n")
    if df_temp.shape[0] == 0:
        missing.append(segment)

In [ ]:
len(missing)

In [ ]:
for seg in missing:
    print(seg)

Check segments to see which have similar travel time:

In [ ]:
df.head(5)

In [ ]:
for segment in segments:
    print(str(segment[0]) + " to " + str(segment[1]) + ",",  end='')
    df_temp = df.loc[(df['stoppointid_first'] == segment[0]) & (df['stoppointid_next'] == segment[1])]
    print(str(df_temp.shape[0]) + ",", end = '')
    print(str(df_temp['time_diff'].mean()) + ",", end = '')
    print(str(df_temp['time_diff'].median()))

# 10. Plot Features

## 10.1 Histograms for the Continuous Features

In [ ]:
df[continuous_columns].hist(figsize=(30,30))

## 10.2 Box Plots for the Continuous Features

In [ ]:
for col in continuous_columns:
    f = df[col].plot(kind='box', figsize=(10,5))
    plt.show()

## 10.3 Bar Charts for the Categorical Features

In [ ]:
# select the features to plot
# some categorical features will not be plotted here due to the high number of values e.g. bus stop no.
categorical_columns_plot = df[['lineid', 'routeid', 'direction',  \
                          'month', 'day', 'hour', 'day_of_week', 'weekday', 'bank_holiday']].columns

In [ ]:
for column in categorical_columns_plot:
    f = df[column].value_counts().plot(kind='bar', title=column, figsize=(8,6))
    plt.show()

## 10.4 Correlation Matrix for Continuous Features

In [ ]:
# Correlation matrix using code found on https://stanford.edu/~mwaskom/software/seaborn/examples/many_pairwise_correlations.html
sns.set(style="white")

# Calculate correlation of all pairs of continuous features
corr = df[continuous_columns].corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(10, 10))

# Generate a custom colormap - blue and red
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, annot=True, mask=mask, cmap=cmap, vmax=1, vmin=-1,
            square=True, xticklabels=True, yticklabels=True,
            linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)
plt.yticks(rotation = 0)
plt.xticks(rotation = 90)

We are mainly interested here in the correlation between the weather data and *time_diff* or *actualtime_arr_stop_next*. The correlation for these features with the weather data is quite low.

Some of the weather features have very high correlation with each other e.g. temp with wetb and vappr with dewpt. The weather features we can use will be dependent on what is available in the weather forecast we receive. However, if all features are available in the forecast we may need to remove some of the features due to high correlation.

## 10.5 Correlations between the Continuous Features & the Target Feature

We have two options for our target feature:

- actualtime_arr_stop_next
- time_diff

I will plot scatter plots for each.

In [ ]:
# print a scatter plot for each continuous feature against actualtime_arr_stop_next
for col in continuous_columns:
    df.plot.scatter(x=col, y='actualtime_arr_stop_next', c='purple')

In [ ]:
# print a scatter plot for each continuous feature against time_diff
for col in continuous_columns:
    df.plot.scatter(x=col, y='time_diff', c='purple')

## 10.6 Correlations between the Categorical Features & the Target Feature

We have two options for our target feature:

- actualtime_arr_stop_next
- time_diff

I will plot box plots for each.

In [ ]:
for col in categorical_columns_plot:
    df.boxplot(column=['actualtime_arr_stop_next'], by=[col], flierprops=flierprops, figsize=(8,5))

In [ ]:
for col in categorical_columns_plot:
    df.boxplot(column=['time_diff'], by=[col], flierprops=flierprops, figsize=(8,5))

# 11. Deal with Outliers

To be investigated...

# 12. Feature Selection for the Initial Model

For an initial model, I will use the following features:

- stoppointid_first
- actualtime_arr_stop_first
- stoppointid_next
- actualtime_arr_stop_next (the target feature)
- month
- day_of_week             
- weekday
- bank_holiday
- rain
- temp
- rhum
- msl

The following features have not been used for this initial model:

- dayofservice: the relevant information from this feature has been split into other features e.g. month, day_of_week
- tripid: we will not be able to provide this information to the model
- lineid: constant as the initial model just covers one route
- routeid: we may not be able to map this to the timetable information we have and if not won't be able to provide it to the model
- direction: could look at including in a future model
- progrnumber_first: probably shouldn't use as routes can change over time so progrnumber for a stop may not be the same as what the model was trained on
- progrnumber_next: as above
- day: could be included in a future model, but seems to have less correlation with other features compared to day_of_week
- hour: could be included in a future model
- record_date: the relevant information from this feature has been split into other features e.g. month, day_of_week
- wetb: not available from OpenWeather
- dewpt: not available from OpenWeather
- vappr: not available from OpenWeather
- time_diff: could look at including in a future model as target feature instead of actualtime_arr_stop_next.

# 13. Prepare Categorical Features

We will use binary encoding to avoid wrongly assigning an order to the categorical features where one isn't really present. The following features will be encoded:

- stoppointid_first
- stoppointid_next
- month
- day_of_week

**weekday** and **bank_holiday** are already binary so don't need to be encoded.

In [ ]:
# encode values for stoppointid_first
df_dummies = pd.get_dummies(df['stoppointid_first'], prefix='stoppointid_first')
df = pd.concat([df, df_dummies], axis =1)

In [ ]:
# encode values for stoppointid_next
df_dummies = pd.get_dummies(df['stoppointid_next'], prefix='stoppointid_next')
df = pd.concat([df, df_dummies], axis =1)

In [ ]:
# encode values for month
df_dummies = pd.get_dummies(df['month'], prefix='month')
df = pd.concat([df, df_dummies], axis =1)

In [ ]:
# encode values for day_of_week
df_dummies = pd.get_dummies(df['day_of_week'], prefix='day_of_week')
df = pd.concat([df, df_dummies], axis =1)

# 14. Split Test & Training Data

Data is shuffled and split:

In [ ]:
# random_state is set to 2 so that the train and test data will be split the same way each time the notebook is run
df_train, df_test = train_test_split(df, test_size=0.3, random_state=2, shuffle=True)

# 15. Train a Linear Regression Model

Prepare descriptive & target features for the training and test data.

In [ ]:
# Prepare the descriptive & target features for the training data
X_train = df_train[['actualtime_arr_stop_first','rain','temp','rhum','msl','weekday','bank_holiday','stoppointid_first_340.0','stoppointid_first_350.0','stoppointid_first_351.0','stoppointid_first_352.0','stoppointid_first_353.0','stoppointid_first_395.0','stoppointid_first_396.0','stoppointid_first_397.0','stoppointid_first_398.0','stoppointid_first_399.0','stoppointid_first_400.0','stoppointid_first_407.0','stoppointid_first_1016.0','stoppointid_first_1017.0','stoppointid_first_1018.0','stoppointid_first_1019.0','stoppointid_first_1020.0','stoppointid_first_1069.0','stoppointid_first_1070.0','stoppointid_first_1071.0','stoppointid_first_1072.0','stoppointid_first_1076.0','stoppointid_first_1077.0','stoppointid_first_1078.0','stoppointid_first_1079.0','stoppointid_first_1080.0','stoppointid_first_1081.0','stoppointid_first_1082.0','stoppointid_first_1083.0','stoppointid_first_1085.0','stoppointid_first_1086.0','stoppointid_first_1087.0','stoppointid_first_1088.0','stoppointid_first_1089.0','stoppointid_first_1090.0','stoppointid_first_1091.0','stoppointid_first_1092.0','stoppointid_first_1093.0','stoppointid_first_1094.0','stoppointid_first_1095.0','stoppointid_first_1096.0','stoppointid_first_1101.0','stoppointid_first_1102.0','stoppointid_first_1103.0','stoppointid_first_1105.0','stoppointid_first_1107.0','stoppointid_first_1108.0','stoppointid_first_1109.0','stoppointid_first_1110.0','stoppointid_first_1111.0','stoppointid_first_1112.0','stoppointid_first_1113.0','stoppointid_first_1114.0','stoppointid_first_1115.0','stoppointid_first_1117.0','stoppointid_first_1118.0','stoppointid_first_1119.0','stoppointid_first_1120.0','stoppointid_first_1164.0','stoppointid_first_1165.0','stoppointid_first_1166.0','stoppointid_first_1167.0','stoppointid_first_1168.0','stoppointid_first_1169.0','stoppointid_first_1170.0','stoppointid_first_1283.0','stoppointid_first_1285.0','stoppointid_first_1353.0','stoppointid_first_1354.0','stoppointid_first_2437.0','stoppointid_first_2498.0','stoppointid_first_2499.0','stoppointid_first_4528.0','stoppointid_first_7216.0','stoppointid_first_7558.0','stoppointid_first_7577.0','stoppointid_first_7578.0','stoppointid_first_7579.0','stoppointid_first_7581.0','stoppointid_first_7582.0','stoppointid_first_7589.0','stoppointid_next_340.0','stoppointid_next_350.0','stoppointid_next_351.0','stoppointid_next_352.0','stoppointid_next_353.0','stoppointid_next_354.0','stoppointid_next_396.0','stoppointid_next_397.0','stoppointid_next_398.0','stoppointid_next_399.0','stoppointid_next_400.0','stoppointid_next_407.0','stoppointid_next_1016.0','stoppointid_next_1017.0','stoppointid_next_1018.0','stoppointid_next_1019.0','stoppointid_next_1020.0','stoppointid_next_1069.0','stoppointid_next_1070.0','stoppointid_next_1071.0','stoppointid_next_1072.0','stoppointid_next_1076.0','stoppointid_next_1077.0','stoppointid_next_1078.0','stoppointid_next_1079.0','stoppointid_next_1080.0','stoppointid_next_1081.0','stoppointid_next_1082.0','stoppointid_next_1083.0','stoppointid_next_1085.0','stoppointid_next_1086.0','stoppointid_next_1087.0','stoppointid_next_1088.0','stoppointid_next_1089.0','stoppointid_next_1090.0','stoppointid_next_1091.0','stoppointid_next_1092.0','stoppointid_next_1093.0','stoppointid_next_1094.0','stoppointid_next_1095.0','stoppointid_next_1096.0','stoppointid_next_1101.0','stoppointid_next_1102.0','stoppointid_next_1103.0','stoppointid_next_1104.0','stoppointid_next_1107.0','stoppointid_next_1108.0','stoppointid_next_1109.0','stoppointid_next_1110.0','stoppointid_next_1111.0','stoppointid_next_1112.0','stoppointid_next_1113.0','stoppointid_next_1114.0','stoppointid_next_1115.0','stoppointid_next_1117.0','stoppointid_next_1118.0','stoppointid_next_1119.0','stoppointid_next_1120.0','stoppointid_next_1164.0','stoppointid_next_1165.0','stoppointid_next_1166.0','stoppointid_next_1167.0','stoppointid_next_1168.0','stoppointid_next_1169.0','stoppointid_next_1170.0','stoppointid_next_1283.0','stoppointid_next_1285.0','stoppointid_next_1353.0','stoppointid_next_1354.0','stoppointid_next_2437.0','stoppointid_next_2498.0','stoppointid_next_2499.0','stoppointid_next_4528.0','stoppointid_next_7216.0','stoppointid_next_7558.0','stoppointid_next_7577.0','stoppointid_next_7578.0','stoppointid_next_7579.0','stoppointid_next_7581.0','stoppointid_next_7582.0','stoppointid_next_7589.0','month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12','day_of_week_0','day_of_week_1','day_of_week_2','day_of_week_3','day_of_week_4','day_of_week_5','day_of_week_6']]
y_train = df_train.actualtime_arr_stop_next

In [ ]:
# Prepare the descriptive & target features for the test data
X_test = df_test[['actualtime_arr_stop_first','rain','temp','rhum','msl','weekday','bank_holiday','stoppointid_first_340.0','stoppointid_first_350.0','stoppointid_first_351.0','stoppointid_first_352.0','stoppointid_first_353.0','stoppointid_first_395.0','stoppointid_first_396.0','stoppointid_first_397.0','stoppointid_first_398.0','stoppointid_first_399.0','stoppointid_first_400.0','stoppointid_first_407.0','stoppointid_first_1016.0','stoppointid_first_1017.0','stoppointid_first_1018.0','stoppointid_first_1019.0','stoppointid_first_1020.0','stoppointid_first_1069.0','stoppointid_first_1070.0','stoppointid_first_1071.0','stoppointid_first_1072.0','stoppointid_first_1076.0','stoppointid_first_1077.0','stoppointid_first_1078.0','stoppointid_first_1079.0','stoppointid_first_1080.0','stoppointid_first_1081.0','stoppointid_first_1082.0','stoppointid_first_1083.0','stoppointid_first_1085.0','stoppointid_first_1086.0','stoppointid_first_1087.0','stoppointid_first_1088.0','stoppointid_first_1089.0','stoppointid_first_1090.0','stoppointid_first_1091.0','stoppointid_first_1092.0','stoppointid_first_1093.0','stoppointid_first_1094.0','stoppointid_first_1095.0','stoppointid_first_1096.0','stoppointid_first_1101.0','stoppointid_first_1102.0','stoppointid_first_1103.0','stoppointid_first_1105.0','stoppointid_first_1107.0','stoppointid_first_1108.0','stoppointid_first_1109.0','stoppointid_first_1110.0','stoppointid_first_1111.0','stoppointid_first_1112.0','stoppointid_first_1113.0','stoppointid_first_1114.0','stoppointid_first_1115.0','stoppointid_first_1117.0','stoppointid_first_1118.0','stoppointid_first_1119.0','stoppointid_first_1120.0','stoppointid_first_1164.0','stoppointid_first_1165.0','stoppointid_first_1166.0','stoppointid_first_1167.0','stoppointid_first_1168.0','stoppointid_first_1169.0','stoppointid_first_1170.0','stoppointid_first_1283.0','stoppointid_first_1285.0','stoppointid_first_1353.0','stoppointid_first_1354.0','stoppointid_first_2437.0','stoppointid_first_2498.0','stoppointid_first_2499.0','stoppointid_first_4528.0','stoppointid_first_7216.0','stoppointid_first_7558.0','stoppointid_first_7577.0','stoppointid_first_7578.0','stoppointid_first_7579.0','stoppointid_first_7581.0','stoppointid_first_7582.0','stoppointid_first_7589.0','stoppointid_next_340.0','stoppointid_next_350.0','stoppointid_next_351.0','stoppointid_next_352.0','stoppointid_next_353.0','stoppointid_next_354.0','stoppointid_next_396.0','stoppointid_next_397.0','stoppointid_next_398.0','stoppointid_next_399.0','stoppointid_next_400.0','stoppointid_next_407.0','stoppointid_next_1016.0','stoppointid_next_1017.0','stoppointid_next_1018.0','stoppointid_next_1019.0','stoppointid_next_1020.0','stoppointid_next_1069.0','stoppointid_next_1070.0','stoppointid_next_1071.0','stoppointid_next_1072.0','stoppointid_next_1076.0','stoppointid_next_1077.0','stoppointid_next_1078.0','stoppointid_next_1079.0','stoppointid_next_1080.0','stoppointid_next_1081.0','stoppointid_next_1082.0','stoppointid_next_1083.0','stoppointid_next_1085.0','stoppointid_next_1086.0','stoppointid_next_1087.0','stoppointid_next_1088.0','stoppointid_next_1089.0','stoppointid_next_1090.0','stoppointid_next_1091.0','stoppointid_next_1092.0','stoppointid_next_1093.0','stoppointid_next_1094.0','stoppointid_next_1095.0','stoppointid_next_1096.0','stoppointid_next_1101.0','stoppointid_next_1102.0','stoppointid_next_1103.0','stoppointid_next_1104.0','stoppointid_next_1107.0','stoppointid_next_1108.0','stoppointid_next_1109.0','stoppointid_next_1110.0','stoppointid_next_1111.0','stoppointid_next_1112.0','stoppointid_next_1113.0','stoppointid_next_1114.0','stoppointid_next_1115.0','stoppointid_next_1117.0','stoppointid_next_1118.0','stoppointid_next_1119.0','stoppointid_next_1120.0','stoppointid_next_1164.0','stoppointid_next_1165.0','stoppointid_next_1166.0','stoppointid_next_1167.0','stoppointid_next_1168.0','stoppointid_next_1169.0','stoppointid_next_1170.0','stoppointid_next_1283.0','stoppointid_next_1285.0','stoppointid_next_1353.0','stoppointid_next_1354.0','stoppointid_next_2437.0','stoppointid_next_2498.0','stoppointid_next_2499.0','stoppointid_next_4528.0','stoppointid_next_7216.0','stoppointid_next_7558.0','stoppointid_next_7577.0','stoppointid_next_7578.0','stoppointid_next_7579.0','stoppointid_next_7581.0','stoppointid_next_7582.0','stoppointid_next_7589.0','month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12','day_of_week_0','day_of_week_1','day_of_week_2','day_of_week_3','day_of_week_4','day_of_week_5','day_of_week_6']]
y_test = df_test.actualtime_arr_stop_next

Clear down dataframes that are no longer needed:

In [ ]:
del df
del df_weather
del df_train
del df_test

Train a model using linear regression from scikit-learn:

In [ ]:
linreg = linear_model.LinearRegression().fit(X_train, y_train)

# 16. Test the Model

In [ ]:
# make predictions based on the training data
linreg_predicted = (linreg.predict(X_test))

In [ ]:
print("Mean Absolute Error: ", metrics.mean_absolute_error(y_test, linreg_predicted))
print()
print("Root Mean Squared Error: ", math.sqrt(metrics.mean_squared_error(y_test, linreg_predicted)))
print()

# 17. Pickle the Model

Save the model to a file called **15A_model.sav**:

In [ ]:
filename = '15A_model.sav'
pickle.dump(linreg, open(filename, 'wb'))

# Appendix 1: Data Quality Plan - Bus Data (Before Transformation)

| Feature | Data Quality Issue | Handling Strategy |
|-------------------------|----------------------|------------------------------|
| actualtime_arr_trip | Missing values ~ 5% | This feature is not brought across when data is transformed. |
| actualtime_dep_trip | Missing values ~ 6% | This feature is not brought across when data is transformed. |

# Appendix 2: Data Quality Plan - Bus Data (After Transformation)

| Feature | Data Quality Issue | Handling Strategy |
|-------------------------|----------------------|------------------------------|
| actualtime_arr_stop_first | Missing values ~ 3% | Drop affected rows |
| actualtime_arr_stop_next | Missing values ~ 3% | Drop affected rows |

# Appendix 3: Data Quality Plan - Weather Data

| Feature | Data Quality Issue | Handling Strategy |
|-------------------------|----------------------|------------------------------|
| itemp | Constant feature | Drop feature |
| iwb | Constant feature | Drop feature |
| rain | Missing data - 2 rows | Imputation - replace with 0 after looking at data for other timestamps on the same day |
| irain | Seems to be a missing data indicator | Drop feature as only two rows have missing data, and imputation is performed for these rows. |

# Appendix 4: Data Quality Plan - After Merge & Features Added

| Feature | Data Quality Issue | Handling Strategy |
|-------------------------|----------------------|------------------------------|
| time_diff | For 959 rows, this feature is zero or negative | Drop affected rows|
| All | Missing data for segment 1354 to 7578 due to stop 7589 no longer being on the route. | Decide on an approach - other routes may have data for this segment. If not, we may need to use timetable information or real time info to estimate travel time through this segment.  There is also the possibility of using times between 1354 and 7589 and 7589 and 7578 to approximate travel time, but we would need more info on where stop 7589 was to see if this would be valid to do. |

# Appendix 5: Tests for *transform_data*

In [ ]:
df_test1 = df.loc[5:100]
df_test1 = df_test1.reset_index(drop=True)
df_test1

In [ ]:
df_transformed1 = transform.transform_data(df_test1)
df_transformed1

In [ ]:
pieces = [df[:35], df[42:100]]
df_test2 = pd.concat(pieces)
df_test2 = df_test2.reset_index(drop=True)
df_test2

In [ ]:
df_transformed2 = transform.transform_data(df_test2)
df_transformed2

In [ ]:
pieces = [df[:5], df[10:100]]
df_test3 = pd.concat(pieces)
df_test3 = df_test3.reset_index(drop=True)
df_test3

In [ ]:
df_transformed3 = transform.transform_data(df_test3)
df_transformed3

In [ ]:
pieces = [df[:5], df[8:10], df[14:50]]
df_test4 = pd.concat(pieces)
df_test4 = df_test4.reset_index(drop=True)
df_test4

In [ ]:
df_transformed4 = transform.transform_data(df_test4)
df_transformed4